In [40]:
import pandas as pd

This notebook presents a streamlined view of the data cleaning and shaping process.

# KS4 Data Cleaning and Shaping

The data on KS4 entry and achievement is contained in the SFR01_2016_LA_Tables file.  First it must be read in.

In [41]:
furl = 'EMA_data/key-stage4/Tables/SFR01_2016_LA_Tables.xlsx'
df_gcse = pd.read_excel(furl, "Table LA1")
df_gcse.head(15)

,Table LA1: GCSE and equivalent entries and achievements of pupils at the end of key stage 4 by gender for each local authority1 and region,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,22,Unnamed: 17,All
0,"Year: 2014/152,3 (Revised)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Please select criteria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Boys
1,Coverage: England,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gender:,All,NaN,NaN,NaN,NaN,NaN,NaN,Girls
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Number of pupils at the end of key stage 4,Percentage of pupils at the end of key stage 4...,NaN,NaN,NaN,NaN,Average capped4 GCSE and equivalents point sco...,Average GCSE and equivalents point score per p...,A*-C in English and mathematics GCSEs includin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Region/\nLocal Authority1,Region/\nLocal Authority number,NaN,5+A*-C grades,5+ A*-C including English & mathematics GCSEs,5+A*-G grades,5+ A*-G including English & mathematics GCSEs,Any passes,NaN,NaN,Per cent entered,Per cent achieved,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Total (state-funded sector)1,NaN,550786,66.5,57.3,94.3,91.7,98.1,313.5,373.2,96,59.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,England1,E92000001,611081,64.9,53.8,91,85.7,97.9,306.5,361.7,90,55.8,NaN,NaN,NaN,,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The data requires cleaning.  

In [42]:
#Extra rows and columns are removed.
df_gcse = df_gcse.iloc[3:182, 0:12]

#required columns are renamed to less verbose versions
df_gcse.rename(columns={
    'Table LA1: GCSE and equivalent entries and achievements of pupils at the end of key stage 4 by gender for each local authority1 and region': 'Region', 
    'Unnamed: 1': 'Region #',
    'Unnamed: 3': '% 5+ A*-C',
    'Unnamed: 4': '% 5+ A*-C inc Eng & Maths GCSE',
    'Unnamed: 5': '% 5+ A*-G',     
    'Unnamed: 6': '% 5+ A*-G inc Eng & Maths GCSE', 
    'Unnamed: 10': 'GCSE or Equv. Maths & English % entered',
    'Unnamed: 11': 'GCSE or Equv. Maths & English A*-C % achieved'
}, inplace=True)

#required columns are selected
df_gcse = df_gcse[["Region", 
         "Region #", 
         "% 5+ A*-C", 
         "% 5+ A*-C inc Eng & Maths GCSE", 
         "% 5+ A*-G", 
         "% 5+ A*-G inc Eng & Maths GCSE",
         "GCSE or Equv. Maths & English % entered",
        "GCSE or Equv. Maths & English A*-C % achieved"]]

#rows with null region names are removed
df_gcse = df_gcse[df_gcse['Region'].notnull()]

#an extra row of former heading data is removed
df_gcse.drop(df_gcse[df_gcse["Region #"]=='Region/\nLocal Authority number'].index, inplace=True)

#rows with null region codes are removed
df_gcse = df_gcse[df_gcse['Region #'].notnull()]

#rows with totals over larger regions are removed
df_gcse.drop(df_gcse[df_gcse["Region #"].str.contains('E9.*')].index, inplace=True)
df_gcse.drop(df_gcse[df_gcse["Region #"].str.contains('E12.*')].index, inplace=True)
df_gcse.drop(df_gcse[df_gcse["Region #"].str.contains('E13.*')].index, inplace=True)

#non-matching region codes are changed to closest match in the deprivation data
df_gcse.replace('E06000048','E06000057', inplace=True)
df_gcse.replace('E08000020','E08000037', inplace=True)

#data is unavailable for the city of london, so drop the row
df_gcse.drop(df_gcse[df_gcse["Region #"]=='E09000001'].index, inplace=True)

#types must be properly cast
df_gcse = df_gcse.astype({'Region': 'string',
                  'Region #': 'string',
                  '% 5+ A*-C': 'float64',
                  '% 5+ A*-C inc Eng & Maths GCSE': 'float64',
                  '% 5+ A*-G': 'float64',
                  '% 5+ A*-G inc Eng & Maths GCSE': 'float64',
                  'GCSE or Equv. Maths & English % entered': 'float64',
                  'GCSE or Equv. Maths & English A*-C % achieved': 'float64'})


#display
df_gcse

,Region,Region #,% 5+ A*-C,% 5+ A*-C inc Eng & Maths GCSE,% 5+ A*-G,% 5+ A*-G inc Eng & Maths GCSE,GCSE or Equv. Maths & English % entered,GCSE or Equv. Maths & English A*-C % achieved
11,County Durham,E06000047,65.1,55.1,93.3,91.1,95.6,57.6
12,Darlington,E06000005,62.2,52.5,93.6,90.2,95.5,55.9
13,Gateshead,E08000037,67.6,58.1,92.6,90.1,95.4,60.2
14,Hartlepool,E06000001,63.7,53.4,94.0,90.8,95.7,55.8
15,Middlesbrough,E06000002,60.5,46.1,92.0,88.6,93.3,48.1
...,...,...,...,...,...,...,...,...
177,Somerset,E10000027,68.0,58.6,95.7,92.7,96.5,60.6
178,South Gloucestershire,E06000025,62.6,52.9,94.0,92.2,96.5,56.0
179,Swindon,E06000030,62.0,53.0,95.3,92.7,96.3,55.4
180,Torbay,E06000027,63.9,57.3,91.3,89.1,95.1,62.0


Finally add the achievement excluding English and Maths columns.

In [66]:
df_gcse['% 5+ A*-G excl Eng & Maths GCSE'] = df_gcse['% 5+ A*-G'] - df_gcse['% 5+ A*-G inc Eng & Maths GCSE']
df_gcse['% 5+ A*-C excl Eng & Maths GCSE'] = df_gcse['% 5+ A*-C'] - df_gcse['% 5+ A*-C inc Eng & Maths GCSE']
df_gcse

,Region,Region #,% 5+ A*-C,% 5+ A*-C inc Eng & Maths GCSE,% 5+ A*-G,% 5+ A*-G inc Eng & Maths GCSE,GCSE or Equv. Maths & English % entered,GCSE or Equv. Maths & English A*-C % achieved,% 5+ A*-G excl Eng & Maths GCSE,% 5+ A*-C excl Eng & Maths GCSE
11,County Durham,E06000047,65.1,55.1,93.3,91.1,95.6,57.6,2.2,10.0
12,Darlington,E06000005,62.2,52.5,93.6,90.2,95.5,55.9,3.4,9.7
13,Gateshead,E08000037,67.6,58.1,92.6,90.1,95.4,60.2,2.5,9.5
14,Hartlepool,E06000001,63.7,53.4,94.0,90.8,95.7,55.8,3.2,10.3
15,Middlesbrough,E06000002,60.5,46.1,92.0,88.6,93.3,48.1,3.4,14.4
...,...,...,...,...,...,...,...,...,...,...
177,Somerset,E10000027,68.0,58.6,95.7,92.7,96.5,60.6,3.0,9.4
178,South Gloucestershire,E06000025,62.6,52.9,94.0,92.2,96.5,56.0,1.8,9.7
179,Swindon,E06000030,62.0,53.0,95.3,92.7,96.3,55.4,2.6,9.0
180,Torbay,E06000027,63.9,57.3,91.3,89.1,95.1,62.0,2.2,6.6


## Deprivation Data Cleaning

The data on Deprivation scores at the correct local authority level is contained in the File_11_ID_2015_Upper-tier_Local_Authority_Summaries file. First it must be read in.  The needed data is spread across multiple sheets, and each series is added in with a merge.

In [43]:
furl = 'EMA_data/Deprivation_Index/File_11_ID_2015_Upper-tier_Local_Authority_Summaries.xlsx'

#read in the first sheet
df_deprivation2 = pd.read_excel(furl, "IMD")[['Upper Tier Local Authority District code (2013)', 'Upper Tier Local Authority District name (2013)', 'IMD - Average score', 'IMD - Extent']]
temp = pd.read_excel(furl, "Income")[['Upper Tier Local Authority District code (2013)', 'Income - Average score']]

#merge in subsequent sheets
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl, "Employment")[['Upper Tier Local Authority District code (2013)', 'Employment - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl, "Education")[['Upper Tier Local Authority District code (2013)', 'Education, Skills and Training - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl, "Health")[['Upper Tier Local Authority District code (2013)', 'Health Deprivation and Disability - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl, "Crime")[['Upper Tier Local Authority District code (2013)', 'Crime - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl, "Barriers")[['Upper Tier Local Authority District code (2013)', 'Barriers to Housing and Services - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)
temp = pd.read_excel(furl, "Living")[['Upper Tier Local Authority District code (2013)', 'Living Environment - Average score']]
df_deprivation2 = df_deprivation2.merge(temp)

#display
df_deprivation2

,Upper Tier Local Authority District code (2013),Upper Tier Local Authority District name (2013),IMD - Average score,IMD - Extent,Income - Average score,Employment - Average score,"Education, Skills and Training - Average score",Health Deprivation and Disability - Average score,Crime - Average score,Barriers to Housing and Services - Average score,Living Environment - Average score
0,E06000001,Hartlepool,33.178,0.4589,0.239,0.211,30.510,0.929,0.078,13.128,8.260
1,E06000002,Middlesbrough,40.216,0.5604,0.268,0.216,40.640,1.195,0.633,18.316,12.907
2,E06000003,Redcar and Cleveland,28.567,0.3394,0.204,0.186,27.875,0.752,-0.013,12.863,7.923
3,E06000004,Stockton-on-Tees,24.625,0.2799,0.177,0.156,24.637,0.548,-0.263,17.844,7.068
4,E06000005,Darlington,23.639,0.2438,0.168,0.150,22.569,0.502,0.224,12.346,10.710
...,...,...,...,...,...,...,...,...,...,...,...
147,E10000029,Suffolk,18.314,0.1052,0.114,0.098,25.229,-0.297,-0.303,24.098,21.130
148,E10000030,Surrey,9.386,0.0070,0.070,0.060,11.608,-1.030,-0.375,20.292,13.362
149,E10000031,Warwickshire,15.006,0.0598,0.102,0.087,17.029,-0.263,-0.186,20.485,18.075
150,E10000032,West Sussex,14.027,0.0418,0.097,0.084,18.235,-0.517,-0.328,21.771,14.708


The only changes required are a few name changes to match the previous dataframe.

In [44]:
df_deprivation2.rename(columns={'Upper Tier Local Authority District code (2013)': 'Region #',
                              'Upper Tier Local Authority District name (2013)': 'Region'}, inplace=True)
df_deprivation2.head()

,Region #,Region,IMD - Average score,IMD - Extent,Income - Average score,Employment - Average score,"Education, Skills and Training - Average score",Health Deprivation and Disability - Average score,Crime - Average score,Barriers to Housing and Services - Average score,Living Environment - Average score
0,E06000001,Hartlepool,33.178,0.4589,0.239,0.211,30.510,0.929,0.078,13.128,8.260
1,E06000002,Middlesbrough,40.216,0.5604,0.268,0.216,40.640,1.195,0.633,18.316,12.907
2,E06000003,Redcar and Cleveland,28.567,0.3394,0.204,0.186,27.875,0.752,-0.013,12.863,7.923
3,E06000004,Stockton-on-Tees,24.625,0.2799,0.177,0.156,24.637,0.548,-0.263,17.844,7.068
4,E06000005,Darlington,23.639,0.2438,0.168,0.150,22.569,0.502,0.224,12.346,10.710


# Combining KS4 and Deprivation Data

We should expect 151 rows and 19 columns in the merged dataframe.

In [68]:
df_merged = df_gcse.merge(df_deprivation2)
df_merged

,Region,Region #,% 5+ A*-C,% 5+ A*-C inc Eng & Maths GCSE,% 5+ A*-G,% 5+ A*-G inc Eng & Maths GCSE,GCSE or Equv. Maths & English % entered,GCSE or Equv. Maths & English A*-C % achieved,% 5+ A*-G excl Eng & Maths GCSE,% 5+ A*-C excl Eng & Maths GCSE,IMD - Average score,IMD - Extent,Income - Average score,Employment - Average score,"Education, Skills and Training - Average score",Health Deprivation and Disability - Average score,Crime - Average score,Barriers to Housing and Services - Average score,Living Environment - Average score
0,County Durham,E06000047,65.1,55.1,93.3,91.1,95.6,57.6,2.2,10.0,25.741,0.2646,0.178,0.168,23.093,0.766,-0.172,17.605,6.077
1,Darlington,E06000005,62.2,52.5,93.6,90.2,95.5,55.9,3.4,9.7,23.639,0.2438,0.168,0.150,22.569,0.502,0.224,12.346,10.710
2,Gateshead,E08000037,67.6,58.1,92.6,90.1,95.4,60.2,2.5,9.5,25.932,0.2735,0.185,0.165,26.145,0.779,-0.301,18.009,8.557
3,Hartlepool,E06000001,63.7,53.4,94.0,90.8,95.7,55.8,3.2,10.3,33.178,0.4589,0.239,0.211,30.510,0.929,0.078,13.128,8.260
4,Middlesbrough,E06000002,60.5,46.1,92.0,88.6,93.3,48.1,3.4,14.4,40.216,0.5604,0.268,0.216,40.640,1.195,0.633,18.316,12.907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Somerset,E10000027,68.0,58.6,95.7,92.7,96.5,60.6,3.0,9.4,17.783,0.0796,0.113,0.100,20.030,-0.263,-0.397,25.496,24.111
147,South Gloucestershire,E06000025,62.6,52.9,94.0,92.2,96.5,56.0,1.8,9.7,11.358,0.0104,0.087,0.074,17.479,-0.857,-0.294,18.358,11.164
148,Swindon,E06000030,62.0,53.0,95.3,92.7,96.3,55.4,2.6,9.0,17.857,0.1459,0.125,0.099,26.089,-0.135,0.073,17.064,13.737
149,Torbay,E06000027,63.9,57.3,91.3,89.1,95.1,62.0,2.2,6.6,28.788,0.3104,0.191,0.176,24.462,0.568,0.306,15.990,27.731


# Acquiring Region Border Data

Data for applying region shapes to a folium map is available from https://geoportal.statistics.gov.uk.  It will be requested from 2 different APIs, one for E0 level areas and one for E1 level areas.

In [48]:
import requests

In [49]:
def addRegion(col, region):
    endpoint = ('https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Local_Authority_Districts_December_2018_Boundaries_GB_BFC/FeatureServer/0/query?where=LAD18CD%3D\'' 
            + 
           region
            +'\'')
    params = {'f' : 'pgeojson' }

    try:
        r = requests.get(endpoint, params=params)
        r.raise_for_status()
        col[region] = r.json()
        if col[region].get('type') != 'FeatureCollection':
            print("No usable data at: " + endpoint)
            del col[region]
            return 0
        return 1
    except requests.exceptions.RequestException as e:
        print(e)
        return 0


In [50]:
def addCounty(col, region):
    endpoint = ('https://opendata.arcgis.com/datasets/0b2c8acda82d4dd3aecdad8233d6bd9b_0.geojson?where=cty19cd%3D\'' 
            + 
           region
            +'\'')
    params = {'f' : 'pgeojson' }

    try:
        r = requests.get(endpoint, params=params)
        r.raise_for_status()
        col[region] = r.json()
        if col[region].get('type') != 'FeatureCollection':
            del col[region]
            print("No usable data at: " + endpoint)
            return 0
        return 1
    except requests.exceptions.RequestException as e:
        print(e)
        return 0

In [51]:
def addAllRegions(col, regions):
    suc = 0
    for re in regions:  
        if re.find("E1") == -1:
            suc += addRegion(col, re)
        else:
            suc += addCounty(col, re)
    print(str(suc) + ' regions added of ' + str(len(regions)) + '.')

Note that the full process will take a while to run, and E10000009 will not be found.

In [53]:
mapdata = {}
addAllRegions(mapdata, df_merged['Region #'])
len(mapdata)

HTTPSConnectionPool(host='opendata.arcgis.com', port=443): Max retries exceeded with url: /datasets/0b2c8acda82d4dd3aecdad8233d6bd9b_0.geojson?where=cty19cd%3D'E10000009'&f=pgeojson (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response',)))
150 regions added of 151.


150

I had to find dorset from the same data file another way and import it manually:

In [72]:
dorset = { "type": "Feature", "properties": { "objectid": 6, "cty18cd": "E10000009", "cty18nm": "Dorset", "bng_e": 370871, "bng_n": 99796, "long": -2.41466999, "lat": 50.79697037, "st_areashape": 2541994583.6881537, "st_lengthshape": 549084.96642460523 }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -2.316805819489742, 51.07984338821079 ], [ -2.320545570945493, 51.080991412836212 ], [ -2.325852759830444, 51.079675588105218 ], [ -2.326532624085067, 51.078910205074081 ], [ -2.328472029348099, 51.078016330734144 ], [ -2.329957273857268, 51.076711893056249 ], [ -2.334372120890082, 51.07526155272128 ], [ -2.337743281378781, 51.074603531172954 ], [ -2.342184252621335, 51.075017712871336 ], [ -2.343670280326549, 51.074640195354036 ], [ -2.347087162075585, 51.072685108423535 ], [ -2.3476385767203, 51.071801319439558 ], [ -2.349116353049518, 51.070726792160194 ], [ -2.349978119439732, 51.068800796693147 ], [ -2.349414621892978, 51.066590406930004 ], [ -2.349457303978249, 51.064118314873816 ], [ -2.349939072346706, 51.063658313239856 ], [ -2.349521704630521, 51.063005836757 ], [ -2.349621014175198, 51.061811358472518 ], [ -2.346508568517646, 51.058578968174338 ], [ -2.344486292508463, 51.055482625530843 ], [ -2.342194902076708, 51.056067568252317 ], [ -2.340108758249186, 51.056062869876229 ], [ -2.333185457288363, 51.051494053666865 ], [ -2.333266998545819, 51.051121560008127 ], [ -2.332636917947913, 51.050590090214726 ], [ -2.332544710619131, 51.049478064810067 ], [ -2.333313900546315, 51.046737719568625 ], [ -2.33144816064418, 51.044375332717884 ], [ -2.330367050541944, 51.043794786092434 ], [ -2.329629639765005, 51.042191766601469 ], [ -2.330076496439348, 51.042052964003702 ], [ -2.330014421250103, 51.041344574622883 ], [ -2.332307750499611, 51.03970960125227 ], [ -2.335419209807716, 51.039610725451034 ], [ -2.33607765459184, 51.039934349872588 ], [ -2.336545703134569, 51.038182266892171 ], [ -2.337727954506824, 51.036992760952927 ], [ -2.338921073569724, 51.036912894461821 ], [ -2.341536285193909, 51.035480868122519 ], [ -2.342755186802345, 51.035221004451699 ], [ -2.343410241402587, 51.033359479138383 ], [ -2.344254022961379, 51.032518980421116 ], [ -2.347292011314964, 51.031814010790121 ], [ -2.347527389350397, 51.031246782872266 ], [ -2.349768799443225, 51.030078269824841 ], [ -2.353469395791343, 51.02783706694251 ], [ -2.356770016854727, 51.026843296391469 ], [ -2.360674113435683, 51.02517496422378 ], [ -2.360459319199742, 51.023407717106643 ], [ -2.361564170479738, 51.02301765746374 ], [ -2.363477983191357, 51.019985792273296 ], [ -2.367248675689778, 51.017778989502105 ], [ -2.369097276589472, 51.015925238764517 ], [ -2.369146803783664, 51.014275914934053 ], [ -2.371161838407303, 51.013460189320377 ], [ -2.378047145670197, 51.011714351667848 ], [ -2.378157786162769, 51.011115098677955 ], [ -2.375913378881841, 51.010849855131134 ], [ -2.376262966425734, 51.009669818123577 ], [ -2.375655458431259, 51.009277031095444 ], [ -2.376472950428492, 51.009202484881726 ], [ -2.377138913749633, 51.008356830104539 ], [ -2.377846894805114, 51.008127912490473 ], [ -2.377676059438351, 51.007279648093743 ], [ -2.378539658508297, 51.007257977934621 ], [ -2.379548479892696, 51.006527165546686 ], [ -2.379640281604915, 51.006241801626786 ], [ -2.379019170081182, 51.005935375908621 ], [ -2.379687499998494, 51.005217441461717 ], [ -2.381348386598102, 51.004922477425417 ], [ -2.381755804266709, 51.001946516189427 ], [ -2.380910098385869, 51.001184870150595 ], [ -2.381498660768116, 51.00030896324207 ], [ -2.381003979120391, 50.999935519910295 ], [ -2.380946252467234, 50.996711052688468 ], [ -2.378877106978698, 50.994776325817135 ], [ -2.379384976248207, 50.994325007880008 ], [ -2.378512917595689, 50.994157026161794 ], [ -2.378236400359559, 50.993438552279073 ], [ -2.377456884727513, 50.993611877295237 ], [ -2.377841188077765, 50.992670928134132 ], [ -2.377227745940099, 50.992401325025334 ], [ -2.377030458488163, 50.991801259278283 ], [ -2.37775968624669, 50.991413152692182 ], [ -2.376911765621395, 50.991215414150929 ], [ -2.37681808280556, 50.990729184574477 ], [ -2.376322207380842, 50.990903475691006 ], [ -2.374631631672679, 50.990456532427011 ], [ -2.374056360770459, 50.990677816154609 ], [ -2.373568173557439, 50.990390711199638 ], [ -2.372928375001416, 50.990740750164733 ], [ -2.372296496677549, 50.990480168684151 ], [ -2.371619488653244, 50.991336626351107 ], [ -2.371020493512512, 50.991266608828852 ], [ -2.370606827645679, 50.991751663629763 ], [ -2.369795552018998, 50.991505122757694 ], [ -2.368683374184425, 50.991757740927092 ], [ -2.3675422702829, 50.991247924277758 ], [ -2.365997819695655, 50.991430773448045 ], [ -2.364874217196324, 50.990419065821293 ], [ -2.361236222084057, 50.991265802081472 ], [ -2.354949896414564, 50.990896537194537 ], [ -2.354842224438285, 50.990224244059284 ], [ -2.352691360744357, 50.989926813232479 ], [ -2.351941294024437, 50.988713289203204 ], [ -2.352466889285637, 50.988499543126132 ], [ -2.352291722192013, 50.988139489060487 ], [ -2.351483320313709, 50.987877533396635 ], [ -2.350385260582712, 50.988117266562234 ], [ -2.346983134160563, 50.985262449875684 ], [ -2.345850987877135, 50.984733518948289 ], [ -2.344021135364174, 50.98483871227819 ], [ -2.34385249024576, 50.984380546708636 ], [ -2.343905326744696, 50.983820233061529 ], [ -2.344630804326925, 50.983493470319885 ], [ -2.343875526746558, 50.982295191812447 ], [ -2.344766557355789, 50.981805135288958 ], [ -2.344483399778437, 50.981267388331794 ], [ -2.344930818888049, 50.980320064842054 ], [ -2.344134576092378, 50.979755873896529 ], [ -2.344556536867631, 50.978829302923849 ], [ -2.345915547894675, 50.979041131284738 ], [ -2.351182623238502, 50.978582029867596 ], [ -2.353207151336193, 50.977884424321438 ], [ -2.358261630639236, 50.97741755442938 ], [ -2.362556678444662, 50.976563565898367 ], [ -2.365565161152962, 50.975501112814442 ], [ -2.374782379085121, 50.973225629297005 ], [ -2.381072954552874, 50.972811457297446 ], [ -2.383541774412838, 50.972359197018989 ], [ -2.392737693110214, 50.969241575309844 ], [ -2.397037195239968, 50.968401528317422 ], [ -2.403138859315965, 50.965173977107661 ], [ -2.414927619336176, 50.960583438300986 ], [ -2.416213008673523, 50.961657123865592 ], [ -2.418346401036052, 50.962327536194344 ], [ -2.419441921933103, 50.96364639004392 ], [ -2.419596625723387, 50.965169174520653 ], [ -2.421343451661063, 50.967164590694281 ], [ -2.424949149232341, 50.970270150317155 ], [ -2.427991475358969, 50.971386757595226 ], [ -2.431138143622253, 50.970677414161699 ], [ -2.431766733080325, 50.971336932821877 ], [ -2.433894110741866, 50.970228342426651 ], [ -2.43666157606512, 50.969332364822023 ], [ -2.441361076799811, 50.968394798907568 ], [ -2.441453133214861, 50.968642573008616 ], [ -2.447177766359896, 50.967850222792229 ], [ -2.444849448983585, 50.963986065133 ], [ -2.44475668235221, 50.963061086677619 ], [ -2.442876070841917, 50.962966543017245 ], [ -2.442270968185976, 50.96252284208321 ], [ -2.444335978415026, 50.961060900894317 ], [ -2.443665735829029, 50.961247826856756 ], [ -2.442375639644994, 50.960795844145117 ], [ -2.443478303659981, 50.958462652461421 ], [ -2.444754773772803, 50.958081939576708 ], [ -2.445734383559333, 50.955850779186584 ], [ -2.448604869764284, 50.952912690015452 ], [ -2.452485919375768, 50.950273765489364 ], [ -2.455364616493698, 50.949267141938073 ], [ -2.457407662448052, 50.949582001639342 ], [ -2.457993561469273, 50.949960973846466 ], [ -2.458663435613677, 50.949747085810444 ], [ -2.460420911261288, 50.951128590625764 ], [ -2.457906031629579, 50.952787661965594 ], [ -2.459029453501082, 50.953522502630939 ], [ -2.458643974572828, 50.954288337259143 ], [ -2.461445908055421, 50.956309584527887 ], [ -2.463808352078044, 50.955346143888661 ], [ -2.466817330974556, 50.959810712016385 ], [ -2.470242191089037, 50.959151341751173 ], [ -2.473025979568225, 50.958030401674598 ], [ -2.474742330617719, 50.957951529605268 ], [ -2.475452347664496, 50.960002486719311 ], [ -2.476415533579896, 50.96076388057071 ], [ -2.474577832213089, 50.962507753622752 ], [ -2.475460131408581, 50.962862958597299 ], [ -2.475920351530178, 50.963585945520947 ], [ -2.480194418366317, 50.965865137510811 ], [ -2.485086974651904, 50.971143340196022 ], [ -2.487779470968272, 50.97311315589004 ], [ -2.490345249224013, 50.972772406629431 ], [ -2.490483793312836, 50.974163910685832 ], [ -2.488217908441658, 50.974452170184094 ], [ -2.488332701450951, 50.975756481395806 ], [ -2.492140933244319, 50.978517372867785 ], [ -2.497175708931463, 50.98735260588024 ], [ -2.502231844946704, 50.993762237607136 ], [ -2.508534196785104, 50.994658490297446 ], [ -2.510860059561818, 50.994427106353967 ], [ -2.512811707862526, 50.993984263856504 ], [ -2.516285699109662, 50.994499537524462 ], [ -2.518140878329785, 50.993334886852978 ], [ -2.518462134883123, 50.993780340501338 ], [ -2.517298187280832, 50.994433029322977 ], [ -2.518052765022516, 50.994501604415575 ], [ -2.520102011338358, 50.994009568405943 ], [ -2.52351630628795, 50.992249856677489 ], [ -2.526530531953628, 50.991925169143265 ], [ -2.526863131632688, 50.992111555474956 ], [ -2.526466857040016, 50.992597171044359 ], [ -2.529362768141505, 50.992641614196891 ], [ -2.535561463413182, 50.994022402237029 ], [ -2.535854949587599, 50.996856425972865 ], [ -2.535167307387778, 50.99937293882612 ], [ -2.547435118328291, 50.99954168729731 ], [ -2.546922190516038, 50.997251954996017 ], [ -2.547291374898746, 50.996418394560614 ], [ -2.548535913149618, 50.995986341333811 ], [ -2.551489940475148, 50.996344682529518 ], [ -2.551924346467795, 50.993697987465239 ], [ -2.547375545145251, 50.992209586092194 ], [ -2.54561907837085, 50.992120727856481 ], [ -2.544724507033409, 50.991666247670558 ], [ -2.545427928897752, 50.989807808580593 ], [ -2.545423436128968, 50.985080501058306 ], [ -2.548091974494248, 50.984852130741253 ], [ -2.550217902374952, 50.983801710038229 ], [ -2.553333285260721, 50.982827462210466 ], [ -2.555128583066972, 50.981423327866082 ], [ -2.556207025571729, 50.979535157581083 ], [ -2.560101349286534, 50.978694550926974 ], [ -2.562371402075645, 50.977710636112576 ], [ -2.569104852744354, 50.985370071078862 ], [ -2.574630368498902, 50.986693681358226 ], [ -2.574349850338832, 50.985093511223965 ], [ -2.574938961326556, 50.983318154428503 ], [ -2.576875232011733, 50.982305936523687 ], [ -2.578336339935741, 50.982151167600698 ], [ -2.579358856563444, 50.981555269254812 ], [ -2.580823496019106, 50.981347501077437 ], [ -2.583181113073082, 50.978863672450956 ], [ -2.584256925403475, 50.978890646019529 ], [ -2.586727425988416, 50.977804563947082 ], [ -2.586023366372804, 50.976869289097792 ], [ -2.589291629784048, 50.976920228464998 ], [ -2.591549973683343, 50.978761250081256 ], [ -2.595845219046929, 50.981339970113204 ], [ -2.603387171098203, 50.976344469462298 ], [ -2.602695171766798, 50.976035110618902 ], [ -2.602749292474717, 50.975600527433109 ], [ -2.6020674730167, 50.974979970545881 ], [ -2.601075836518093, 50.974628100361024 ], [ -2.59897680584486, 50.974549819498208 ], [ -2.59802189623315, 50.973510723180489 ], [ -2.598950896990288, 50.972655233526488 ], [ -2.598560228112109, 50.971521542701403 ], [ -2.599582939541323, 50.970760858859798 ], [ -2.599223603897723, 50.970061312980306 ], [ -2.599651439210892, 50.969557382349926 ], [ -2.599882678282781, 50.96740157191978 ], [ -2.599082452548092, 50.966637678967189 ], [ -2.597256639179006, 50.965746889939119 ], [ -2.597355686566708, 50.96481385494981 ], [ -2.595997574827418, 50.96462213318609 ], [ -2.594908613059974, 50.962109795333461 ], [ -2.595465149061305, 50.960869574839471 ], [ -2.596725389437596, 50.961210227136071 ], [ -2.597945528636396, 50.96075258474059 ], [ -2.59837803518897, 50.959506693178298 ], [ -2.598731838136198, 50.958566953376987 ], [ -2.59814890535158, 50.957750696839518 ], [ -2.599554520835583, 50.954352380205627 ], [ -2.598556284423514, 50.953013159562964 ], [ -2.599299175016172, 50.952656845757012 ], [ -2.599354886792815, 50.952234751347959 ], [ -2.59848194983678, 50.951880450734237 ], [ -2.598658943063665, 50.950594520201449 ], [ -2.597988500268672, 50.950155566419497 ], [ -2.599204128810047, 50.949259040983996 ], [ -2.598159958609725, 50.948982031149164 ], [ -2.598200272475843, 50.948574487774088 ], [ -2.599192413885435, 50.947680008751888 ], [ -2.60168424316112, 50.947406366137372 ], [ -2.601688573988544, 50.94685957202681 ], [ -2.604472708295862, 50.945970252041434 ], [ -2.604960416403289, 50.945377796932789 ], [ -2.605574804039957, 50.945332352410432 ], [ -2.606750668471933, 50.945892693408851 ], [ -2.607174467871368, 50.945757434034967 ], [ -2.607305709123625, 50.944029231880208 ], [ -2.608144050825274, 50.944023959482422 ], [ -2.608620201700897, 50.943648298303202 ], [ -2.608611426972186, 50.942211381998696 ], [ -2.611588177667738, 50.940193090409736 ], [ -2.613926369502713, 50.940283368356006 ], [ -2.614004316810125, 50.941337755071423 ], [ -2.615733264545262, 50.941204568820332 ], [ -2.616035377128687, 50.941448399126791 ], [ -2.615919057231185, 50.939748553521738 ], [ -2.614896136255912, 50.938377226240426 ], [ -2.613016669981214, 50.937334939772775 ], [ -2.610359899587491, 50.936704125208763 ], [ -2.61052090831113, 50.936508198168823 ], [ -2.606710343357656, 50.935021829185359 ], [ -2.604629142395479, 50.932556979120321 ], [ -2.604032354552853, 50.930975584183173 ], [ -2.604472708575774, 50.927725205321693 ], [ -2.604040002200761, 50.926724702451992 ], [ -2.605199452321318, 50.926600016084791 ], [ -2.606593718869206, 50.925831129139731 ], [ -2.609781101607289, 50.925300976157281 ], [ -2.613408980625933, 50.923552604807789 ], [ -2.614156874489507, 50.923619775782178 ], [ -2.614805141961958, 50.923256574354596 ], [ -2.615188353668926, 50.922435379199932 ], [ -2.614630229574594, 50.922186514341426 ], [ -2.614904152258643, 50.92192067151138 ], [ -2.614531110100908, 50.921134933138163 ], [ -2.615340400713602, 50.919935525347057 ], [ -2.615190714376359, 50.919480340921893 ], [ -2.616045480445146, 50.919242082077467 ], [ -2.61530460656666, 50.917646219396666 ], [ -2.616074031100884, 50.917529687276293 ], [ -2.61562580404417, 50.916653460073768 ], [ -2.616133369470182, 50.915990727054741 ], [ -2.616994162166911, 50.915672328309654 ], [ -2.616434712666796, 50.915104332418927 ], [ -2.617044481818364, 50.914314204388774 ], [ -2.616620680629514, 50.913350646915646 ], [ -2.617336408697199, 50.913267744234112 ], [ -2.617335805149891, 50.9123612798507 ], [ -2.618872216811577, 50.911622056134803 ], [ -2.61941590364495, 50.911754017056332 ], [ -2.620024024634852, 50.911491777805367 ], [ -2.619995967972898, 50.910553095617225 ], [ -2.62043406416576, 50.91002016981583 ], [ -2.622097937978057, 50.910108477968727 ], [ -2.622399933590985, 50.908754310483587 ], [ -2.62406503860158, 50.907866891587339 ], [ -2.623776829618162, 50.907166941022687 ], [ -2.622354820688693, 50.906116146808024 ], [ -2.622213456468526, 50.905433476463024 ], [ -2.619977717992692, 50.904680073690535 ], [ -2.619327866373645, 50.90404325818556 ], [ -2.618947762493196, 50.902612800556966 ], [ -2.619662317701879, 50.902339208352828 ], [ -2.618697974939886, 50.901361450046579 ], [ -2.617632501609133, 50.90155589599717 ], [ -2.61752060074985, 50.900079958400894 ], [ -2.617000828245518, 50.899599764626899 ], [ -2.616204605718198, 50.899493400010527 ], [ -2.616210345479799, 50.89853295815972 ], [ -2.615635468949191, 50.898068332307368 ], [ -2.615528217913932, 50.897269499035332 ], [ -2.616381643358276, 50.896746978493432 ], [ -2.615480886345182, 50.896373167032074 ], [ -2.615705931802232, 50.895862059440979 ], [ -2.614989186899292, 50.894449543672323 ], [ -2.615404930280998, 50.894273749864411 ], [ -2.614535077559251, 50.893758621689479 ], [ -2.614845769003727, 50.89327942856054 ], [ -2.612559393263267, 50.892188112741351 ], [ -2.610300969708074, 50.892358186366337 ], [ -2.609759000011439, 50.891907830600481 ], [ -2.609442457660782, 50.890854584089851 ], [ -2.610364598551446, 50.890158237277291 ], [ -2.610152416838464, 50.888937229077058 ], [ -2.611610641235375, 50.886510520049391 ], [ -2.611227531517232, 50.886130359085186 ], [ -2.611759475680915, 50.885613242800865 ], [ -2.611238529696448, 50.885017951146601 ], [ -2.612318822756543, 50.884791034513064 ], [ -2.612912006746148, 50.885039730111835 ], [ -2.613099022446452, 50.884446116862257 ], [ -2.614505215533794, 50.883708532561421 ], [ -2.614950380235434, 50.883833845728581 ], [ -2.615179046854213, 50.882525111042057 ], [ -2.616101294889519, 50.881633541157115 ], [ -2.617895247730715, 50.880999050372765 ], [ -2.620521362881735, 50.881262985634578 ], [ -2.620937187412136, 50.880892084925343 ], [ -2.622186435228252, 50.880760370682211 ], [ -2.622662643457241, 50.880234511068103 ], [ -2.62300637907511, 50.88032167716311 ], [ -2.623328960622048, 50.879569995214702 ], [ -2.624129366402474, 50.87925905833972 ], [ -2.627181148157598, 50.87989464439751 ], [ -2.627961010305901, 50.886305795979297 ], [ -2.630787522629905, 50.885802277298644 ], [ -2.637600005613834, 50.886957580934499 ], [ -2.640767681025087, 50.886976189232819 ], [ -2.642512548051467, 50.886702286237359 ], [ -2.643024652247736, 50.887238061347929 ], [ -2.647104592928285, 50.888352211186344 ], [ -2.646990711108038, 50.887823148170945 ], [ -2.646152746209016, 50.887376399354117 ], [ -2.647359507400202, 50.885406638890345 ], [ -2.646834620957024, 50.88415327656638 ], [ -2.649576579415696, 50.884134450378482 ], [ -2.65041939461201, 50.883307793319872 ], [ -2.658426418346282, 50.886603718364761 ], [ -2.660611475808957, 50.887104799986957 ], [ -2.661412850769381, 50.886366494850215 ], [ -2.664458405725886, 50.885097427337442 ], [ -2.666317385874672, 50.88357155805128 ], [ -2.668861383892776, 50.882444670733655 ], [ -2.671792675124407, 50.880581639301518 ], [ -2.678965739588885, 50.87747992659321 ], [ -2.680565248030029, 50.877123508687966 ], [ -2.684628454027642, 50.877001675856434 ], [ -2.695726909059057, 50.875990952360524 ], [ -2.696503846269479, 50.875479188763826 ], [ -2.695684646197999, 50.874680983734734 ], [ -2.697134766107277, 50.873478086001178 ], [ -2.699578029411925, 50.872823197052128 ], [ -2.702903270047326, 50.873268198297211 ], [ -2.704203801612025, 50.872518496394335 ], [ -2.707145132502526, 50.869766011713949 ], [ -2.711211155458209, 50.870059658219915 ], [ -2.711761164869694, 50.870897155233408 ], [ -2.714389780864044, 50.871143614919617 ], [ -2.713783876319244, 50.866646494029929 ], [ -2.714420887542778, 50.86546722117528 ], [ -2.71455590879055, 50.863894459966666 ], [ -2.717996623066207, 50.864025378728691 ], [ -2.721385360945089, 50.864930752122874 ], [ -2.723091646537485, 50.866190871639184 ], [ -2.724032105979372, 50.866084264919692 ], [ -2.724989331992301, 50.865405679500462 ], [ -2.729912036809473, 50.864704163253435 ], [ -2.733145878493405, 50.864957295818236 ], [ -2.737066229648106, 50.864232998139023 ], [ -2.738239850252041, 50.86467248671633 ], [ -2.750453307674968, 50.863806007280509 ], [ -2.753829427243468, 50.864327357804477 ], [ -2.754824329523463, 50.865485520677709 ], [ -2.754988771242281, 50.866778506478369 ], [ -2.756962573848343, 50.866607499580958 ], [ -2.757498939537377, 50.866902585643679 ], [ -2.759916644564365, 50.865981294090332 ], [ -2.760956773196188, 50.863183152640694 ], [ -2.760968569296161, 50.86129099738163 ], [ -2.761687947506487, 50.860799757116084 ], [ -2.767396762484398, 50.860697648131683 ], [ -2.771699029567049, 50.860268239855337 ], [ -2.775412868856247, 50.860683345866811 ], [ -2.780061501772622, 50.860318754381353 ], [ -2.782286052554023, 50.859696851952023 ], [ -2.784080902950539, 50.856838595087467 ], [ -2.786199839824928, 50.854513220386558 ], [ -2.787538022729938, 50.853750635143392 ], [ -2.788977673886875, 50.853443243169004 ], [ -2.792432257209344, 50.85283439087118 ], [ -2.793720445806383, 50.853398497113041 ], [ -2.795074423620351, 50.854653675972962 ], [ -2.799053074841899, 50.855449298147029 ], [ -2.800416179604376, 50.856152171052798 ], [ -2.801142525516488, 50.856170653270972 ], [ -2.801687281626868, 50.856788322233335 ], [ -2.802904223461523, 50.857242145745509 ], [ -2.80784207175352, 50.858679322870358 ], [ -2.80705225477808, 50.858936516342595 ], [ -2.805606170949988, 50.861509525967818 ], [ -2.805590053230216, 50.861892709255656 ], [ -2.806499334337552, 50.862050946597975 ], [ -2.807809505703668, 50.862878239044541 ], [ -2.807320057539908, 50.863253012090119 ], [ -2.80806844592912, 50.863630902298091 ], [ -2.810151611979889, 50.862567928791442 ], [ -2.811425094010112, 50.863409818916551 ], [ -2.812937096627532, 50.862107942248606 ], [ -2.812374894377143, 50.861234146447053 ], [ -2.813900557366946, 50.859094906447403 ], [ -2.817729950491686, 50.855614866651379 ], [ -2.82047660982177, 50.853935513015607 ], [ -2.822165987770416, 50.85346403046892 ], [ -2.823158837019755, 50.852479511930589 ], [ -2.824419974352463, 50.852058757934273 ], [ -2.826320587115045, 50.850681121268735 ], [ -2.827156822091487, 50.850650919833221 ], [ -2.827427297329495, 50.851241549707659 ], [ -2.828121490560988, 50.851219541348762 ], [ -2.828492162439705, 50.851931004968002 ], [ -2.828525810883946, 50.851589046064412 ], [ -2.828958237573644, 50.85170014975251 ], [ -2.829376331500542, 50.850600004942208 ], [ -2.828483060991789, 50.848772780883365 ], [ -2.828956931463113, 50.848659642369142 ], [ -2.829641217855409, 50.848556782574143 ], [ -2.831000376420066, 50.849897809936479 ], [ -2.831590327283584, 50.849765043808596 ], [ -2.831444642868243, 50.849397389968281 ], [ -2.837158014674408, 50.848754879596335 ], [ -2.838847368870234, 50.848060191603473 ], [ -2.842845940950953, 50.847346018388734 ], [ -2.843160011168474, 50.84877275001314 ], [ -2.843768602946589, 50.849195473450663 ], [ -2.843491434988461, 50.849582443223824 ], [ -2.843980443068704, 50.85023177605909 ], [ -2.844737452028957, 50.850141693619975 ], [ -2.844925897649089, 50.851139414420913 ], [ -2.845394717269118, 50.851214319789669 ], [ -2.845195553621263, 50.8515943430641 ], [ -2.846344272006082, 50.851329695672419 ], [ -2.84659551674429, 50.850766700089245 ], [ -2.847308371878029, 50.850831733168256 ], [ -2.847611650187467, 50.851337567493601 ], [ -2.848555453288744, 50.851212927256917 ], [ -2.848820465514496, 50.850629148519495 ], [ -2.848998771848231, 50.850902142552002 ], [ -2.850690889325974, 50.85075755770584 ], [ -2.852502602981005, 50.849380139779953 ], [ -2.853672646493405, 50.849660252095333 ], [ -2.854027076316188, 50.84931413869311 ], [ -2.854417614106402, 50.849380501196769 ], [ -2.854974166424245, 50.849596770809967 ], [ -2.855355936847705, 50.850355622826108 ], [ -2.856061373599281, 50.850015017485418 ], [ -2.857999178288033, 50.850407217372421 ], [ -2.85831260622684, 50.850068584297553 ], [ -2.860325814892442, 50.849776840853899 ], [ -2.861927299342558, 50.848957413607266 ], [ -2.862928339138007, 50.849083061265993 ], [ -2.863470758125428, 50.848607864623929 ], [ -2.864087614253306, 50.848859605548029 ], [ -2.864488270532843, 50.848477964302305 ], [ -2.865091559464863, 50.848762164867821 ], [ -2.865768256199911, 50.848481139243113 ], [ -2.866019077766728, 50.849066429338791 ], [ -2.866943363973235, 50.849347336344145 ], [ -2.8675059527624, 50.848888153386781 ], [ -2.867938370443083, 50.849146639730144 ], [ -2.869640687167039, 50.848870445296711 ], [ -2.869230517242407, 50.848429207143845 ], [ -2.869887124993295, 50.848140215475624 ], [ -2.871168956902828, 50.848249312044466 ], [ -2.871433417793993, 50.849022475716595 ], [ -2.872536878365787, 50.848929710402352 ], [ -2.872692607358057, 50.849435775208015 ], [ -2.873360920610811, 50.849466730377074 ], [ -2.874323061072984, 50.848881262838972 ], [ -2.876259041223249, 50.849251595585606 ], [ -2.876788619711159, 50.849085788460606 ], [ -2.877024626687961, 50.849323153719141 ], [ -2.877185617406392, 50.848739282967315 ], [ -2.877907020714175, 50.848653748283809 ], [ -2.877965879377788, 50.848987832023248 ], [ -2.878382728230041, 50.848944234995464 ], [ -2.87889314781495, 50.848363930980724 ], [ -2.877876097624754, 50.847461602349931 ], [ -2.878879415562236, 50.847177949673927 ], [ -2.879054633993941, 50.846449046534289 ], [ -2.880582474536281, 50.846724391309642 ], [ -2.880928541713809, 50.847220915342767 ], [ -2.880652303251805, 50.847565621987791 ], [ -2.881108075003763, 50.847779831662763 ], [ -2.880857105372477, 50.847955241043351 ], [ -2.881993793977109, 50.84819216856296 ], [ -2.882096869773217, 50.848614919526867 ], [ -2.882865722247139, 50.84833210769694 ], [ -2.883037033454802, 50.848826352832859 ], [ -2.882603581027172, 50.849268451414453 ], [ -2.883439546109269, 50.84953101599811 ], [ -2.883420651038927, 50.84920652061281 ], [ -2.883921823209542, 50.849646120435253 ], [ -2.884887155236756, 50.849681948170165 ], [ -2.884911045092363, 50.850269909792772 ], [ -2.88618014390079, 50.850598391478371 ], [ -2.886277064018591, 50.850162345622742 ], [ -2.887120424521633, 50.850368190039788 ], [ -2.890093023286035, 50.849653995250222 ], [ -2.890314145200193, 50.849031797829866 ], [ -2.892470702522462, 50.848371421294466 ], [ -2.892458745785675, 50.847744711826138 ], [ -2.893191221444356, 50.847572690835293 ], [ -2.893535381677378, 50.846772436173595 ], [ -2.894345114663239, 50.847297639522935 ], [ -2.895009714832114, 50.846694595455801 ], [ -2.895753874845243, 50.84720141394105 ], [ -2.89650399715479, 50.847058993767774 ], [ -2.897344258679682, 50.847543486358333 ], [ -2.898514657478976, 50.846799784310917 ], [ -2.899470550638892, 50.847309531253799 ], [ -2.899687994556116, 50.846801520559978 ], [ -2.901034545490204, 50.846577958922303 ], [ -2.900651853405866, 50.845954115305403 ], [ -2.901620301019501, 50.845942131911656 ], [ -2.902551745837136, 50.844813565395711 ], [ -2.901289054789236, 50.844596691664101 ], [ -2.901722460251853, 50.844163514237337 ], [ -2.902816291186093, 50.84409651996922 ], [ -2.903080246664566, 50.843792311557401 ], [ -2.906471994722843, 50.8437111403355 ], [ -2.906746877683889, 50.844262084213362 ], [ -2.908458507538509, 50.844118396791899 ], [ -2.908936899127821, 50.844611918742594 ], [ -2.910223065186745, 50.844793464186161 ], [ -2.910560911356519, 50.843979632893955 ], [ -2.911114861905553, 50.843907876275054 ], [ -2.911048373929347, 50.844432709132121 ], [ -2.912127125024806, 50.844103228713088 ], [ -2.913260795405262, 50.842442296767786 ], [ -2.913886990651423, 50.842431095175044 ], [ -2.91425650334566, 50.84200736187227 ], [ -2.916497450638614, 50.841773005714415 ], [ -2.917304659396573, 50.841796323771227 ], [ -2.917002838981679, 50.841615296528033 ], [ -2.917369533091843, 50.841551277337047 ], [ -2.917705398313693, 50.841802249182876 ], [ -2.91924259262642, 50.841822499354258 ], [ -2.919090367295752, 50.841525133124605 ], [ -2.920123797312293, 50.841493603656176 ], [ -2.920747052058121, 50.840613655527662 ], [ -2.921673640465849, 50.840212467193986 ], [ -2.922066799120295, 50.840410740058935 ], [ -2.922348433667562, 50.84007218448366 ], [ -2.924969608318103, 50.839968699884288 ], [ -2.925205027757904, 50.839449709653231 ], [ -2.926303008837098, 50.839883490448386 ], [ -2.927296527522745, 50.839707380807795 ], [ -2.927527685597462, 50.840049960263578 ], [ -2.928621935699721, 50.839863238097749 ], [ -2.927877057080942, 50.840761188951333 ], [ -2.929028032204029, 50.840434572427398 ], [ -2.930071072336691, 50.840958658310747 ], [ -2.930268659609689, 50.839110833962465 ], [ -2.930669936720861, 50.838720957409294 ], [ -2.931799521435987, 50.838958355371588 ], [ -2.93283601234769, 50.838801634742673 ], [ -2.93248999281294, 50.838191085216899 ], [ -2.934554136580419, 50.837648528699503 ], [ -2.934381628192767, 50.836474519446327 ], [ -2.938025762750075, 50.835127806916681 ], [ -2.939715873759899, 50.836411852670878 ], [ -2.941682385894448, 50.835895938660563 ], [ -2.942083465709858, 50.835007871123985 ], [ -2.943019208644264, 50.83479073843597 ], [ -2.944260209970907, 50.833233906958682 ], [ -2.943823848422279, 50.832648395582453 ], [ -2.944441379208376, 50.832289983830499 ], [ -2.944114655122269, 50.831932927980382 ], [ -2.944643400696597, 50.832025738726713 ], [ -2.944608630177659, 50.831358759528065 ], [ -2.943873914901266, 50.830201904055379 ], [ -2.944707727057666, 50.829655637996723 ], [ -2.944098739209745, 50.82917042057359 ], [ -2.944713238415614, 50.828733815919421 ], [ -2.945387593316477, 50.828936922395229 ], [ -2.945171482257707, 50.828502554813618 ], [ -2.946031754070184, 50.828839095221817 ], [ -2.946130747889791, 50.826154768668431 ], [ -2.947322627816428, 50.826045293723162 ], [ -2.947058149408796, 50.825320844575579 ], [ -2.948038735921687, 50.82529399191359 ], [ -2.947368764521222, 50.8236915193919 ], [ -2.948218113981425, 50.823284361550044 ], [ -2.949512632041337, 50.823691152499514 ], [ -2.949439591825962, 50.822467803603871 ], [ -2.951482456331199, 50.822880071522199 ], [ -2.952974119962541, 50.82229771513618 ], [ -2.9538542326779, 50.820830123773867 ], [ -2.95432123760533, 50.821172518542291 ], [ -2.95550916709076, 50.820182567184183 ], [ -2.956837233981135, 50.820490916971238 ], [ -2.958171290622921, 50.819987105043936 ], [ -2.95878459594655, 50.81852697291702 ], [ -2.958389540319987, 50.818166023696236 ], [ -2.957628578193684, 50.818301830684469 ], [ -2.95736926896552, 50.81804764599319 ], [ -2.957724094621719, 50.81776414223031 ], [ -2.958092561544263, 50.818010235294771 ], [ -2.958625496683162, 50.817759427483871 ], [ -2.959452761688736, 50.817883878116 ], [ -2.959184304462782, 50.817252088861792 ], [ -2.959616552601053, 50.816800720448853 ], [ -2.961225769204784, 50.816565306446634 ], [ -2.961598887010759, 50.816136894356745 ], [ -2.961346193709007, 50.815720838906806 ], [ -2.960839179148855, 50.815504666911743 ], [ -2.959988681510627, 50.81639749558007 ], [ -2.959058630954349, 50.816036459136861 ], [ -2.958434477837808, 50.816335669151485 ], [ -2.958324323679484, 50.81580866176126 ], [ -2.956613330093377, 50.81585688327943 ], [ -2.954728085812336, 50.815366906943233 ], [ -2.954113868244868, 50.815528459407069 ], [ -2.952733176344319, 50.815064885575154 ], [ -2.951753060644746, 50.815449770139942 ], [ -2.95091602894598, 50.814985329554624 ], [ -2.949830437646788, 50.8150004766837 ], [ -2.949449935732048, 50.815421718721588 ], [ -2.949041408533629, 50.81515344785344 ], [ -2.947961016857933, 50.815425748893638 ], [ -2.946838741335206, 50.814611178613923 ], [ -2.947213018021482, 50.81430956737514 ], [ -2.945675152459871, 50.81407293546728 ], [ -2.944813587440306, 50.8133686079006 ], [ -2.944121470998006, 50.813399390709307 ], [ -2.943202632332593, 50.812388854995831 ], [ -2.942722961916677, 50.812397232295062 ], [ -2.942588804614021, 50.811936033878567 ], [ -2.941505763547275, 50.811725335734018 ], [ -2.941633985242893, 50.811111877271152 ], [ -2.9407396473131, 50.811106514704598 ], [ -2.939786019038112, 50.810479311666505 ], [ -2.939011610272901, 50.81043700677106 ], [ -2.939042805725315, 50.810084244605548 ], [ -2.938401997228367, 50.809834890475834 ], [ -2.936736241581114, 50.810081233036676 ], [ -2.936270835483347, 50.809730664379515 ], [ -2.935523399136001, 50.809831066570787 ], [ -2.934461539467171, 50.809182213167183 ], [ -2.931592257792227, 50.809138660768745 ], [ -2.931277072977747, 50.808636641558834 ], [ -2.930728206596246, 50.809012418402446 ], [ -2.929778448188741, 50.809072234445843 ], [ -2.928956480684906, 50.808424114073105 ], [ -2.927552306838853, 50.808400147996032 ], [ -2.926594967702782, 50.807857465698632 ], [ -2.926251003099096, 50.808263957419314 ], [ -2.925276050422293, 50.807983922056231 ], [ -2.924898861915394, 50.8082908800559 ], [ -2.922722077567706, 50.807594125949706 ], [ -2.921915409611183, 50.80691436772409 ], [ -2.920170649088114, 50.806676339908066 ], [ -2.918556339381158, 50.805630554173874 ], [ -2.917615316421912, 50.805918557779584 ], [ -2.915306740464322, 50.805292826381404 ], [ -2.913592410750145, 50.805296390665887 ], [ -2.91329726026053, 50.805006428113074 ], [ -2.912085673622145, 50.805246161319104 ], [ -2.911295760986961, 50.80462373628076 ], [ -2.907348531725804, 50.804672520299974 ], [ -2.905432689716927, 50.805267480509627 ], [ -2.90260876641041, 50.80461852968444 ], [ -2.900656894151987, 50.804742413924075 ], [ -2.899809376080564, 50.805169797637689 ], [ -2.897188351548138, 50.804522734547852 ], [ -2.895719193588629, 50.804948668441909 ], [ -2.895010603271261, 50.804633064183598 ], [ -2.89391952098719, 50.804943602940583 ], [ -2.893602771336667, 50.804704999404734 ], [ -2.89280364355642, 50.804776752125463 ], [ -2.891559807655016, 50.803776380043566 ], [ -2.890258236059645, 50.803466178555766 ], [ -2.89041569766281, 50.802565665872429 ], [ -2.889879164205148, 50.80218758049358 ], [ -2.890201602275429, 50.801606856149021 ], [ -2.889692965647992, 50.801125975475678 ], [ -2.889809321622272, 50.800679073473589 ], [ -2.888755187601288, 50.79972127050209 ], [ -2.888814003312871, 50.798406058593137 ], [ -2.888303924307709, 50.797476466665131 ], [ -2.887227670550594, 50.796766053120905 ], [ -2.886644344822006, 50.794807333035955 ], [ -2.889301242833791, 50.793873404304478 ], [ -2.887762027888022, 50.793440911535114 ], [ -2.887816009167869, 50.792918874125192 ], [ -2.887039446807005, 50.792522819083935 ], [ -2.890579644627445, 50.7905173986262 ], [ -2.893745186723021, 50.789859177924392 ], [ -2.89563891808698, 50.787538774839192 ], [ -2.897456425114341, 50.786139911291031 ], [ -2.907341793108092, 50.785052622678315 ], [ -2.929652850651553, 50.782432514931457 ], [ -2.931920456082617, 50.781832545495384 ], [ -2.939165874085358, 50.778411871381422 ], [ -2.943545809824955, 50.775870135197756 ], [ -2.954333548297913, 50.767710224970187 ], [ -2.955945392002213, 50.764971196467336 ], [ -2.955706353532178, 50.764164647142586 ], [ -2.956038848077603, 50.763642180680336 ], [ -2.953059616209126, 50.76217102909601 ], [ -2.952591269281835, 50.76138163192924 ], [ -2.949612792607889, 50.760917663596025 ], [ -2.939422812434295, 50.757112518361119 ], [ -2.93506310735221, 50.754266084845639 ], [ -2.936512012902876, 50.752992723400581 ], [ -2.939682947794573, 50.751839530111972 ], [ -2.940572436783302, 50.749892556317803 ], [ -2.940585603807624, 50.748637006722113 ], [ -2.938248545907588, 50.745965046011747 ], [ -2.937862468293218, 50.744096673176315 ], [ -2.938884999544354, 50.743347441968197 ], [ -2.940633896857666, 50.742702929243819 ], [ -2.945654928000306, 50.73781945265933 ], [ -2.946585294182065, 50.737293057898974 ], [ -2.946618501163913, 50.735845752378125 ], [ -2.946307727952604, 50.735261908840407 ], [ -2.945845490733366, 50.735180248438368 ], [ -2.947019301078402, 50.734989919781171 ], [ -2.947778536878882, 50.733780449733196 ], [ -2.949601645464953, 50.732347437645224 ], [ -2.953067353713496, 50.728497068605883 ], [ -2.952602681916686, 50.724594169345679 ], [ -2.948732619625385, 50.721263232289758 ], [ -2.948784694765981, 50.719985780618195 ], [ -2.947761186130257, 50.718264670553332 ], [ -2.939791344811561, 50.719609825320603 ], [ -2.938703764065679, 50.719625781759504 ], [ -2.937916784021198, 50.718809298919091 ], [ -2.937215153989968, 50.719072073211528 ], [ -2.934151784573564, 50.719111948950243 ], [ -2.936956051840797, 50.719223475004171 ], [ -2.936257697358241, 50.719936839304232 ], [ -2.936912017814622, 50.719854263236606 ], [ -2.937754986458472, 50.719006595811024 ], [ -2.938262890234842, 50.719258828653814 ], [ -2.93872620970478, 50.720396328102296 ], [ -2.938266697857326, 50.720865765221546 ], [ -2.937480057384863, 50.721060187785824 ], [ -2.93665508442542, 50.720113548397123 ], [ -2.937424329635636, 50.721246741102775 ], [ -2.936900833919623, 50.721640421324345 ], [ -2.937439592826934, 50.721373474842899 ], [ -2.937654312540217, 50.721626271203824 ], [ -2.937102532005766, 50.722586676108349 ], [ -2.935395550651994, 50.723446577060805 ], [ -2.933090602820386, 50.723951616570304 ], [ -2.932561305222558, 50.724338938976814 ], [ -2.932689835419237, 50.724700457875912 ], [ -2.932020188777827, 50.724484417557086 ], [ -2.931003509412498, 50.724958169987055 ], [ -2.929946646589704, 50.724947076724014 ], [ -2.930406791567011, 50.72593084845235 ], [ -2.927625620967947, 50.729159983630836 ], [ -2.920899304375218, 50.73182311886049 ], [ -2.918811537407234, 50.732231702650999 ], [ -2.916664025683818, 50.732119957483896 ], [ -2.914608846408805, 50.732458049777975 ], [ -2.913293884354002, 50.732825418752938 ], [ -2.912268084369317, 50.733633844327883 ], [ -2.910970629478464, 50.733734783814626 ], [ -2.904536306930428, 50.733546588790418 ], [ -2.901139085845474, 50.733073770394739 ], [ -2.894455784320335, 50.732753827442153 ], [ -2.875400945913933, 50.729947733863554 ], [ -2.872102637869693, 50.72912623277351 ], [ -2.861784199511973, 50.72748165705903 ], [ -2.848065231324, 50.724293596442116 ], [ -2.846845987937242, 50.723617207258108 ], [ -2.84271573671543, 50.723354891356863 ], [ -2.840412821477313, 50.722759116475999 ], [ -2.836533941674674, 50.723721382342482 ], [ -2.826932368305724, 50.722495038914005 ], [ -2.818906042514155, 50.720664189224493 ], [ -2.816733275763363, 50.719948307799129 ], [ -2.81378274540234, 50.718406892309481 ], [ -2.811305009437382, 50.717908023411674 ], [ -2.8102310382778, 50.717896605558821 ], [ -2.807014678392185, 50.718762476726184 ], [ -2.805444321500728, 50.718882183950143 ], [ -2.799258375463134, 50.717649608020906 ], [ -2.795394586621648, 50.718188693474445 ], [ -2.793714845841285, 50.718122835779646 ], [ -2.77680226264135, 50.714079602061453 ], [ -2.770359082187442, 50.71184268163325 ], [ -2.767821744266226, 50.711283941225311 ], [ -2.767293770527186, 50.710520305839708 ], [ -2.766999422955379, 50.710882844268646 ], [ -2.765259392695445, 50.710486044902304 ], [ -2.764217106079758, 50.70857466707205 ], [ -2.763860115446037, 50.708572517431392 ], [ -2.764807369127968, 50.710214724158504 ], [ -2.763962875431051, 50.71036777147286 ], [ -2.763834009369041, 50.710106025013758 ], [ -2.763517061609961, 50.710565827405709 ], [ -2.764152326363431, 50.710961915364912 ], [ -2.763902577306204, 50.711540850089044 ], [ -2.761670032315633, 50.71125691625479 ], [ -2.761804630873844, 50.710908044703217 ], [ -2.763039671326754, 50.710996165517869 ], [ -2.763823170181217, 50.709262517647666 ], [ -2.763494246985958, 50.709069484045244 ], [ -2.763483716145406, 50.709465309936277 ], [ -2.762413164755045, 50.709477718471383 ], [ -2.760254748838342, 50.708864141503675 ], [ -2.739905120281257, 50.702522200762296 ], [ -2.731761310784033, 50.698972007583194 ], [ -2.724472446579692, 50.696709110138421 ], [ -2.678138015222379, 50.680171631519208 ], [ -2.652551103263183, 50.670025426965893 ], [ -2.627106868036674, 50.659573347072715 ], [ -2.590872441797832, 50.64287227605498 ], [ -2.544993741930908, 50.619487512244433 ], [ -2.530755055358216, 50.611870523905466 ], [ -2.503997329674398, 50.596550948810147 ], [ -2.497857457341994, 50.593130298570387 ], [ -2.47860631263751, 50.580847418266856 ], [ -2.451498611758425, 50.562272996303101 ], [ -2.449186241426894, 50.560219833629311 ], [ -2.448008710181619, 50.557822423053999 ], [ -2.448015116120283, 50.557156929294436 ], [ -2.449651265878605, 50.555673075465656 ], [ -2.451177452604208, 50.552431544347925 ], [ -2.452106058039944, 50.551525048954382 ], [ -2.451598147141403, 50.549767081500356 ], [ -2.452064008470185, 50.547562025412887 ], [ -2.45271801524782, 50.54658286189747 ], [ -2.45510999415249, 50.544640965425224 ], [ -2.45641134328987, 50.542457782762703 ], [ -2.456101840758693, 50.5413861432272 ], [ -2.454473345854945, 50.540362826345927 ], [ -2.453664475696569, 50.538862362409994 ], [ -2.454401852973774, 50.537464675149508 ], [ -2.454530150090716, 50.535836472058364 ], [ -2.456041168182578, 50.531272941867151 ], [ -2.455394888205708, 50.525686366223702 ], [ -2.456712897108154, 50.523674864661473 ], [ -2.457741137498336, 50.523072805712694 ], [ -2.458681907835194, 50.521891063404432 ], [ -2.459875863475817, 50.519332322692925 ], [ -2.460128413528747, 50.516581301055567 ], [ -2.459690911128869, 50.516113552357382 ], [ -2.459470130017907, 50.51632398974666 ], [ -2.459283699480926, 50.515722192766091 ], [ -2.459439172311121, 50.514293452883329 ], [ -2.457578418691734, 50.513232461365043 ], [ -2.456587039135282, 50.513098758704047 ], [ -2.455064810634651, 50.514671296071796 ], [ -2.454447791631349, 50.514749245913187 ], [ -2.454287683192579, 50.515114087638644 ], [ -2.453522035363568, 50.515135954233038 ], [ -2.453563633943272, 50.515363338301526 ], [ -2.453011684211949, 50.515165811786396 ], [ -2.452609612184279, 50.516186284564959 ], [ -2.452173180326478, 50.516125020281038 ], [ -2.451873707364794, 50.516672085539476 ], [ -2.450691887612036, 50.516535422261526 ], [ -2.4501529047654, 50.517115769297071 ], [ -2.449157718812045, 50.517173555012405 ], [ -2.448904421187936, 50.517985733958355 ], [ -2.448204359660821, 50.518088225344329 ], [ -2.448320662451503, 50.518454730254973 ], [ -2.447446215320173, 50.5186190558696 ], [ -2.447168846774294, 50.519115640262328 ], [ -2.446222645407985, 50.518992448073412 ], [ -2.445295248094166, 50.5196722553599 ], [ -2.444053439052103, 50.519774155136247 ], [ -2.442769159995561, 50.520458029609649 ], [ -2.441552205245131, 50.523534664944336 ], [ -2.439483638208124, 50.524695322441985 ], [ -2.437725938738668, 50.526691241977012 ], [ -2.437177315622128, 50.530228401196439 ], [ -2.434289703526958, 50.531969401835042 ], [ -2.429568529199614, 50.535995983936495 ], [ -2.428262779875279, 50.537974714139651 ], [ -2.425226083908125, 50.538608156228321 ], [ -2.420381291114354, 50.54111229884964 ], [ -2.41820961398214, 50.542731637147952 ], [ -2.417344200201451, 50.547981233424942 ], [ -2.41567517095136, 50.549185066647702 ], [ -2.416343102657279, 50.549860743096211 ], [ -2.415766910043486, 50.550146979282296 ], [ -2.415838750248717, 50.5510927702557 ], [ -2.417312392290902, 50.552377072512378 ], [ -2.418124623143464, 50.554206916896639 ], [ -2.419188320092717, 50.554933285527312 ], [ -2.421552556255072, 50.55838793918636 ], [ -2.421491366706776, 50.559285645640131 ], [ -2.423860412354565, 50.562137675683999 ], [ -2.42673672624341, 50.564845726640286 ], [ -2.428449850555696, 50.565833133802734 ], [ -2.42863588225113, 50.567039304177634 ], [ -2.428238291603741, 50.56741306641873 ], [ -2.42217703291336, 50.569049397342425 ], [ -2.422107174355696, 50.569296974122068 ], [ -2.42844311955324, 50.567738741431008 ], [ -2.431432604757034, 50.568379680144943 ], [ -2.431588454222963, 50.568132678295648 ], [ -2.430713963996771, 50.567616187864999 ], [ -2.433339269497836, 50.567655893973374 ], [ -2.433856333458336, 50.567990295551688 ], [ -2.437361881215987, 50.568251475030358 ], [ -2.437250913600878, 50.56891735316912 ], [ -2.437661344106036, 50.568870859435151 ], [ -2.437360530087124, 50.569024824404039 ], [ -2.438699588255974, 50.569370498867372 ], [ -2.437728048414655, 50.569977583726356 ], [ -2.433711861626483, 50.570099686732711 ], [ -2.431947647803073, 50.571133204942733 ], [ -2.432130336688163, 50.571338494240024 ], [ -2.437835982003006, 50.570192135272634 ], [ -2.438915300022714, 50.56933012811659 ], [ -2.440791889932374, 50.569934568481081 ], [ -2.44115978348811, 50.568654407441443 ], [ -2.441790379196039, 50.568441621161192 ], [ -2.442585507603268, 50.568461079212248 ], [ -2.442602972758512, 50.569419647206296 ], [ -2.443422409020734, 50.569460581960428 ], [ -2.44370203540955, 50.568565613004033 ], [ -2.445023582644554, 50.568702615522042 ], [ -2.445944635527292, 50.568297143588197 ], [ -2.447272042628154, 50.568453059620872 ], [ -2.452847438908932, 50.570449471611589 ], [ -2.45209572476392, 50.570991990145302 ], [ -2.452867832631981, 50.571254293660985 ], [ -2.453433854386692, 50.570483178865246 ], [ -2.455301552205187, 50.571021748717691 ], [ -2.455933405377345, 50.571674858245572 ], [ -2.457508992528629, 50.571053564790155 ], [ -2.459901884705823, 50.571134064767456 ], [ -2.460458828271867, 50.57148258469141 ], [ -2.460435717104365, 50.570995275935566 ], [ -2.462881875705565, 50.572767089236109 ], [ -2.464443122476782, 50.574428117899124 ], [ -2.466151761610472, 50.577280128268221 ], [ -2.468700463530598, 50.580362561944042 ], [ -2.469526079041429, 50.582427590470672 ], [ -2.469853431306095, 50.582252717483819 ], [ -2.469833283933737, 50.582497434832625 ], [ -2.471504482051905, 50.58266238961528 ], [ -2.471032554254783, 50.581093248891762 ], [ -2.469890282132117, 50.579710272654651 ], [ -2.470546481452803, 50.579223776375919 ], [ -2.469769513916826, 50.578643298401325 ], [ -2.469467267183696, 50.577926031062546 ], [ -2.469943817075904, 50.578132720250274 ], [ -2.470381509181064, 50.578830575275369 ], [ -2.472948268163635, 50.579727564669007 ], [ -2.481279861779045, 50.58526727331455 ], [ -2.491877777353026, 50.591439855281706 ], [ -2.492039272550084, 50.591617238037763 ], [ -2.491473273890222, 50.591536021910727 ], [ -2.492778818329232, 50.59235693932834 ], [ -2.496612942184624, 50.594090707384339 ], [ -2.501375037101609, 50.597355481433283 ], [ -2.502571316297189, 50.59757513786186 ], [ -2.517850659404021, 50.606050759317363 ], [ -2.525964563923119, 50.611182749478154 ], [ -2.527873191982241, 50.611742497078069 ], [ -2.528715285801001, 50.61259928331647 ], [ -2.535138332219738, 50.61564556758794 ], [ -2.5360691261813, 50.616451607258647 ], [ -2.538038650608024, 50.617238375396326 ], [ -2.538809983134016, 50.617939871684534 ], [ -2.539939424812127, 50.61831147906922 ], [ -2.54390371366674, 50.62069505345039 ], [ -2.552081490371518, 50.624427360854661 ], [ -2.553580756038455, 50.625568681612478 ], [ -2.556768481599523, 50.626861039279923 ], [ -2.557845923622277, 50.627944095283915 ], [ -2.561042842472647, 50.629149934099175 ], [ -2.566663020434106, 50.632677678463892 ], [ -2.573860934534419, 50.635638875867897 ], [ -2.577725105950887, 50.637909423148983 ], [ -2.611452565701024, 50.654046805112195 ], [ -2.609952445042671, 50.654108640252147 ], [ -2.609660673159058, 50.654410534670824 ], [ -2.607051816472921, 50.654934956962975 ], [ -2.606403119856208, 50.654656881191244 ], [ -2.606159959406184, 50.653681483062947 ], [ -2.604847819795615, 50.654055211482721 ], [ -2.603430273588308, 50.653610270075511 ], [ -2.600286456900109, 50.65378655342704 ], [ -2.599092543580251, 50.654235212782957 ], [ -2.598220950035974, 50.653998673151236 ], [ -2.597467525447069, 50.653265131428356 ], [ -2.597810890899606, 50.651896414378314 ], [ -2.598406872554949, 50.65126384073757 ], [ -2.60072413986255, 50.650839121268895 ], [ -2.598446168915112, 50.649123295711995 ], [ -2.592148672203348, 50.646741785697145 ], [ -2.588548015845852, 50.646096371109969 ], [ -2.58749882076639, 50.645211359705996 ], [ -2.586391851584606, 50.645136007796538 ], [ -2.5861089880836, 50.644339745448391 ], [ -2.582086314986884, 50.641912906409019 ], [ -2.578244936950057, 50.64108585159817 ], [ -2.575998883300088, 50.640027698840598 ], [ -2.574686076816225, 50.638946957455879 ], [ -2.571562189458758, 50.63836520006884 ], [ -2.570258469809739, 50.637562189828031 ], [ -2.56683089873638, 50.638052007700075 ], [ -2.563013754432571, 50.63790145828294 ], [ -2.561584739141954, 50.638196165174548 ], [ -2.561525865178933, 50.638595735224328 ], [ -2.560495868228282, 50.639146560720242 ], [ -2.561371171914524, 50.638549692850958 ], [ -2.560579034933917, 50.638051749621567 ], [ -2.560511020774986, 50.637569118452696 ], [ -2.560143678337771, 50.638787654110274 ], [ -2.560410873877029, 50.637472490476924 ], [ -2.559907622424408, 50.635939798196844 ], [ -2.560099447812258, 50.635186159184094 ], [ -2.5589066547289, 50.633332153267681 ], [ -2.558055490754131, 50.630834341637289 ], [ -2.55544393097304, 50.630353107247558 ], [ -2.554647861858028, 50.630588937200386 ], [ -2.55307897331409, 50.630398536907741 ], [ -2.552311559542723, 50.630667502306643 ], [ -2.549052063520383, 50.629902341067833 ], [ -2.545919715954517, 50.63046023188847 ], [ -2.5452841781843, 50.630887733058607 ], [ -2.546140535942544, 50.629387264192417 ], [ -2.546111286490775, 50.628200287758276 ], [ -2.546765260388939, 50.627668480351225 ], [ -2.549387776811751, 50.628257715530474 ], [ -2.551866435391354, 50.627429420688792 ], [ -2.548847974151274, 50.624885149182525 ], [ -2.547687758067942, 50.624687369364089 ], [ -2.545568517978697, 50.625894342811087 ], [ -2.544648986392072, 50.625237638849633 ], [ -2.544332592826282, 50.623430586931455 ], [ -2.542250257444234, 50.622945649994747 ], [ -2.540244094116752, 50.622070092679579 ], [ -2.539608150054546, 50.62073482169167 ], [ -2.537963377427274, 50.619679481809634 ], [ -2.531686078615165, 50.617465464864388 ], [ -2.524816596725537, 50.616849195615188 ], [ -2.524060383818377, 50.61773752792007 ], [ -2.523448746868459, 50.617902159027857 ], [ -2.522435293593299, 50.617778115471225 ], [ -2.521170986595996, 50.618105745758925 ], [ -2.51749165634176, 50.617857798125968 ], [ -2.516405429620034, 50.616763622712142 ], [ -2.516409556769039, 50.615342705028795 ], [ -2.514576127643176, 50.614289593239477 ], [ -2.514700597514316, 50.611461559924578 ], [ -2.51363667229913, 50.610197364467027 ], [ -2.512909596791318, 50.610142996369355 ], [ -2.512349249229418, 50.610723755721672 ], [ -2.510336724780144, 50.611636431541257 ], [ -2.508381844020101, 50.611222315625035 ], [ -2.507145870432912, 50.610369772469994 ], [ -2.505612741868331, 50.609870134924073 ], [ -2.505009640111936, 50.609114631969319 ], [ -2.505168691089529, 50.608659780545253 ], [ -2.507330201458592, 50.608070334922694 ], [ -2.507831113572763, 50.607604049060242 ], [ -2.506791884243404, 50.606116582543322 ], [ -2.50434045235988, 50.605102935709681 ], [ -2.501390238813869, 50.606857710647262 ], [ -2.501048786886825, 50.60637983965249 ], [ -2.501401226901847, 50.60550324836457 ], [ -2.500672141688089, 50.604056701806655 ], [ -2.499412377418229, 50.603459586065064 ], [ -2.497034296516246, 50.603336642867134 ], [ -2.495698100904491, 50.602855857452681 ], [ -2.495457198119952, 50.602256128132957 ], [ -2.495852251542686, 50.60166448041079 ], [ -2.499005212956703, 50.599982726513701 ], [ -2.499417773382198, 50.598649051596425 ], [ -2.498853708766002, 50.597422993845555 ], [ -2.49806034754349, 50.596286053867473 ], [ -2.495756462660475, 50.594591686638992 ], [ -2.492679797228331, 50.593424858124351 ], [ -2.490276367548199, 50.591908879224817 ], [ -2.487785812449351, 50.591095560081556 ], [ -2.486331857089814, 50.591212293418543 ], [ -2.485474787946838, 50.59166733638255 ], [ -2.484561741111968, 50.591489442431218 ], [ -2.484036504945193, 50.591102244778945 ], [ -2.484184845252281, 50.590146566196324 ], [ -2.483524869058194, 50.58888577744942 ], [ -2.480992403159744, 50.588650722156302 ], [ -2.477024418256963, 50.585738018936098 ], [ -2.47528434633547, 50.585618334483932 ], [ -2.472401683163373, 50.584523901687952 ], [ -2.471838028189323, 50.583536063201095 ], [ -2.470369756568696, 50.583183198075062 ], [ -2.470202311175664, 50.583957262920997 ], [ -2.471343489574108, 50.5846514538554 ], [ -2.47160947375142, 50.585258232223048 ], [ -2.470483244811415, 50.585355460934515 ], [ -2.469980453393347, 50.584937472911463 ], [ -2.469131453365424, 50.586774625846971 ], [ -2.46761117415198, 50.58816479698082 ], [ -2.466226196719666, 50.591816135731378 ], [ -2.464798993934461, 50.593211335641875 ], [ -2.463701042146009, 50.594188744423633 ], [ -2.460629227481729, 50.595021976418472 ], [ -2.460081369082729, 50.595771520738154 ], [ -2.46008941695169, 50.59659434947288 ], [ -2.459228720123276, 50.597581599564542 ], [ -2.452344092274173, 50.59983524346854 ], [ -2.451572399762169, 50.600549590628638 ], [ -2.450627194842474, 50.600706103387466 ], [ -2.449869356127255, 50.601243253305384 ], [ -2.447290741333824, 50.600520254889453 ], [ -2.447584654451556, 50.599779856189613 ], [ -2.447952159594863, 50.599786533836301 ], [ -2.4476208298987, 50.599565741847201 ], [ -2.447299727848926, 50.599538206085668 ], [ -2.446701617130184, 50.600384895019793 ], [ -2.432424425870537, 50.596183300147885 ], [ -2.432795511262889, 50.596595577129754 ], [ -2.442463641945482, 50.599284180342217 ], [ -2.442403232059601, 50.599623446255009 ], [ -2.443304227199857, 50.599572368855384 ], [ -2.448483332483266, 50.601115512313406 ], [ -2.44951729195148, 50.601964069100511 ], [ -2.450906410923105, 50.603742035247265 ], [ -2.450746562738979, 50.604463008295703 ], [ -2.443396674802316, 50.606982297806184 ], [ -2.443045387323491, 50.60749267605533 ], [ -2.443385619289444, 50.608058868284871 ], [ -2.441364047736286, 50.609642139945578 ], [ -2.442840018594524, 50.608647307446105 ], [ -2.443781953224057, 50.609026808170341 ], [ -2.444574909388728, 50.609336743396959 ], [ -2.443904308462656, 50.609854549184327 ], [ -2.44498758523563, 50.609479016243611 ], [ -2.446415981527801, 50.610330615574526 ], [ -2.450462854870112, 50.608630654222253 ], [ -2.452070376758618, 50.610907748625792 ], [ -2.452570477452022, 50.612938235183947 ], [ -2.452472885211755, 50.614106815761332 ], [ -2.451010840318007, 50.61655141840631 ], [ -2.447793607819262, 50.620395771779634 ], [ -2.443868573481686, 50.624196869536711 ], [ -2.440970762196551, 50.626634229126722 ], [ -2.437390535216187, 50.62931682262132 ], [ -2.431764354930121, 50.632653585001613 ], [ -2.425906914671035, 50.635212976418032 ], [ -2.42336640411731, 50.635896721132163 ], [ -2.419652429264246, 50.636455137164546 ], [ -2.418123592365077, 50.635312219515001 ], [ -2.414747166872729, 50.635028434487943 ], [ -2.411079612279011, 50.634101657868541 ], [ -2.410200372723158, 50.633157795532611 ], [ -2.408511811376953, 50.632968601480727 ], [ -2.407955113760135, 50.633830258888075 ], [ -2.406170268954174, 50.634603624149022 ], [ -2.39815549192017, 50.636252756832114 ], [ -2.393615659975973, 50.636677367333803 ], [ -2.384976878131446, 50.635614524378639 ], [ -2.379454217958276, 50.635515700686859 ], [ -2.376438762890595, 50.634361827764565 ], [ -2.376003638257051, 50.633880308548974 ], [ -2.373452388017468, 50.633472145946556 ], [ -2.372575851275929, 50.632955142782428 ], [ -2.366205003828971, 50.631379111897573 ], [ -2.365646215843434, 50.63097708928801 ], [ -2.358319401220347, 50.631533106672805 ], [ -2.355362487301282, 50.630913559449908 ], [ -2.351568267984256, 50.631194883770668 ], [ -2.349271320671164, 50.630909519215862 ], [ -2.343916517354983, 50.632015458823808 ], [ -2.338741306939902, 50.631984762779759 ], [ -2.334463964589431, 50.630956604292294 ], [ -2.333968165902096, 50.62963789000306 ], [ -2.332383314803418, 50.628847457379614 ], [ -2.330127404932233, 50.626915879956456 ], [ -2.327426428408061, 50.626413601630013 ], [ -2.324204911131805, 50.625243611629706 ], [ -2.323302758630243, 50.6248027779098 ], [ -2.320731385626485, 50.625247840119613 ], [ -2.319741992049831, 50.625690332246997 ], [ -2.314757523089976, 50.625325283525726 ], [ -2.313531491878444, 50.62568830179243 ], [ -2.310293150398147, 50.625769863204695 ], [ -2.303380371032409, 50.625165776981909 ], [ -2.298916479705494, 50.624367050746301 ], [ -2.291749542357833, 50.622837521556797 ], [ -2.291271544760054, 50.622571602289668 ], [ -2.291503864467359, 50.62219872478164 ], [ -2.291020695407262, 50.622246684416673 ], [ -2.290597671295359, 50.622881750800673 ], [ -2.281644512173201, 50.622542246788591 ], [ -2.277523726215625, 50.622064729281654 ], [ -2.276344625112874, 50.621312987140868 ], [ -2.277234242378453, 50.621149944232307 ], [ -2.275562824335267, 50.621060316552864 ], [ -2.273940651992111, 50.62118020908359 ], [ -2.274607457711783, 50.621341369071665 ], [ -2.274544783240873, 50.621748032380012 ], [ -2.273071385059071, 50.622161585462308 ], [ -2.271666978752192, 50.621567716348565 ], [ -2.269752214627556, 50.62168455118141 ], [ -2.265557388509229, 50.620820979108352 ], [ -2.262815317875272, 50.619615825010357 ], [ -2.261785463648756, 50.618714330274834 ], [ -2.260115634837692, 50.618134402743209 ], [ -2.259146258408156, 50.618438721623832 ], [ -2.257452581251274, 50.617886680320694 ], [ -2.256984591167761, 50.618110766455139 ], [ -2.256428918632103, 50.617835890379673 ], [ -2.253460612066029, 50.617832486685202 ], [ -2.253165317579002, 50.618121806163686 ], [ -2.251671009588564, 50.61782286819129 ], [ -2.252957074342813, 50.617777837213886 ], [ -2.250362251406384, 50.617330129803797 ], [ -2.249636617837956, 50.616955782014827 ], [ -2.247989599806607, 50.616906249172274 ], [ -2.249776973499966, 50.617836835402002 ], [ -2.249421816569381, 50.618830429984023 ], [ -2.2479234898085, 50.619368722826948 ], [ -2.246043326482092, 50.619344813092731 ], [ -2.244358037090121, 50.618740435150123 ], [ -2.243717808001501, 50.617948574331024 ], [ -2.244871163799473, 50.616857078253361 ], [ -2.245949546090637, 50.61656073737683 ], [ -2.239143587073789, 50.616015480423037 ], [ -2.228692550181916, 50.61603642980522 ], [ -2.228013574670773, 50.616202377632973 ], [ -2.228421753312503, 50.616437174513479 ], [ -2.228262691218926, 50.616857471953814 ], [ -2.227336792306366, 50.617131777029769 ], [ -2.224217921894957, 50.616704312597456 ], [ -2.222800669452038, 50.617096416021617 ], [ -2.223367604046341, 50.617706010255333 ], [ -2.223018481631737, 50.618612261473174 ], [ -2.220405045083129, 50.620155060927587 ], [ -2.218189675922187, 50.620391261487065 ], [ -2.217412798643719, 50.620225402888913 ], [ -2.216664265560849, 50.620670199305025 ], [ -2.213240597555784, 50.620689088067941 ], [ -2.209915041923702, 50.621141140393462 ], [ -2.209131620080065, 50.621412357637887 ], [ -2.208868161348515, 50.621929052116741 ], [ -2.207206169492847, 50.622347482266157 ], [ -2.204776206734515, 50.622054989119484 ], [ -2.204708902746393, 50.621537063642052 ], [ -2.198493786682845, 50.623014608146264 ], [ -2.193143413539088, 50.621743846347783 ], [ -2.187367048814125, 50.619908746373156 ], [ -2.184825784399665, 50.618210408461145 ], [ -2.184534804774943, 50.616835810998779 ], [ -2.186144087957878, 50.616310764813072 ], [ -2.186673412103449, 50.615781102324355 ], [ -2.186555066424802, 50.615166148719332 ], [ -2.187500789341497, 50.614846270527011 ], [ -2.184514068393508, 50.615178376913462 ], [ -2.184458763550789, 50.615837671710551 ], [ -2.183481732415855, 50.6161135152336 ], [ -2.182800427498075, 50.615402331193096 ], [ -2.179246942174021, 50.61462807656806 ], [ -2.173673887821785, 50.61466166905921 ], [ -2.173278138752285, 50.614289954222954 ], [ -2.170991146982618, 50.614269948180443 ], [ -2.170123258621379, 50.613844969115632 ], [ -2.165727968605033, 50.614752417824931 ], [ -2.162766548104047, 50.614663886769108 ], [ -2.16204945963401, 50.61495626906467 ], [ -2.157772572745332, 50.615410007630672 ], [ -2.154364370769358, 50.614016992854758 ], [ -2.151514333756949, 50.612069223280187 ], [ -2.151036024743235, 50.61146191245183 ], [ -2.151902681406425, 50.611119051651301 ], [ -2.149257430183319, 50.611252816212918 ], [ -2.147300789162327, 50.610623981057522 ], [ -2.146893374871915, 50.608688280425689 ], [ -2.14603487208705, 50.608493336238105 ], [ -2.143065817938149, 50.610846020959684 ], [ -2.139323806581916, 50.610514200690623 ], [ -2.137369932203359, 50.61219549697568 ], [ -2.134415794816193, 50.612615364463316 ], [ -2.131355394850396, 50.611443426440374 ], [ -2.129361383974261, 50.60996987881078 ], [ -2.129479456416953, 50.609235011355906 ], [ -2.131479261146532, 50.608241735168839 ], [ -2.131590550290321, 50.607603120310472 ], [ -2.127245710280666, 50.604266969718736 ], [ -2.120866567570081, 50.602503039248809 ], [ -2.11622099884692, 50.599872818296639 ], [ -2.115968441945629, 50.599439624563225 ], [ -2.114538930495569, 50.599039919026957 ], [ -2.114074676460977, 50.599274219380405 ], [ -2.112625898867185, 50.599080532773527 ], [ -2.105068915265122, 50.596876226214093 ], [ -2.103043569251809, 50.597188275785264 ], [ -2.099883252559068, 50.598037247076761 ], [ -2.098094668617606, 50.598118801140579 ], [ -2.090984217131213, 50.59704807475341 ], [ -2.083031223556434, 50.594718522117219 ], [ -2.080816044607888, 50.594770476926485 ], [ -2.078912504824794, 50.595341933014325 ], [ -2.07703076504197, 50.595436739552376 ], [ -2.075686596556583, 50.595084202089424 ], [ -2.074787869934894, 50.593192682169004 ], [ -2.072906110175342, 50.59229547952522 ], [ -2.070280616809473, 50.592830397989644 ], [ -2.067718821966227, 50.592500993710686 ], [ -2.065718657522054, 50.59373325333322 ], [ -2.064442831021922, 50.593670144452297 ], [ -2.06336423178075, 50.593183375617571 ], [ -2.062711243540794, 50.591889619310813 ], [ -2.063974205398125, 50.590826838470711 ], [ -2.064395071629729, 50.588663800428165 ], [ -2.063613423385203, 50.586177633770347 ], [ -2.062082937510343, 50.58457233354946 ], [ -2.061569336234033, 50.582856780990191 ], [ -2.061876635664909, 50.581317009914599 ], [ -2.057658350752809, 50.577138388952633 ], [ -2.055262346695332, 50.576492078802403 ], [ -2.054371332986491, 50.576594996151549 ], [ -2.052481441115843, 50.577620159117274 ], [ -2.042712521501745, 50.579177329676789 ], [ -2.039637238093397, 50.580173952680802 ], [ -2.038594536157308, 50.581469293286325 ], [ -2.037774054398203, 50.581569428244649 ], [ -2.033477893058105, 50.584145480273847 ], [ -2.031922873160389, 50.584448098026222 ], [ -2.028386704507907, 50.587904151609585 ], [ -2.026683638906619, 50.588880298913608 ], [ -2.024424772144979, 50.588873617823005 ], [ -2.022647957917794, 50.589338888675428 ], [ -2.021164560272954, 50.589109924276762 ], [ -2.012673504466368, 50.590751529341915 ], [ -2.009014715883918, 50.590804946296288 ], [ -2.007079387094448, 50.591159405362696 ], [ -2.006284013161663, 50.591612754504702 ], [ -2.00477523503529, 50.591405969336066 ], [ -2.003515190961106, 50.591763040808154 ], [ -1.994860953947102, 50.590807097738022 ], [ -1.991634260393547, 50.590973351474908 ], [ -1.991170975497352, 50.590641484378679 ], [ -1.98986004389599, 50.590551395273401 ], [ -1.988140660797308, 50.590812060019623 ], [ -1.986996354524621, 50.591058366725562 ], [ -1.984337886457829, 50.590505879222221 ], [ -1.980596934473265, 50.590663668728368 ], [ -1.978946804000591, 50.590999699535701 ], [ -1.977197996019283, 50.590726903382027 ], [ -1.966160001344248, 50.591541727496448 ], [ -1.96113546078469, 50.591010551105526 ], [ -1.960987308973023, 50.590795565024102 ], [ -1.960321642326362, 50.591205398564725 ], [ -1.959530669536629, 50.591031617576256 ], [ -1.958300706779838, 50.592027600776859 ], [ -1.957273587196098, 50.59202184268586 ], [ -1.956261228403664, 50.592925279951423 ], [ -1.953578714195628, 50.59413562902467 ], [ -1.951286918893835, 50.594300134853917 ], [ -1.95056141321711, 50.594993172227063 ], [ -1.95202388408868, 50.596198873595512 ], [ -1.953179266291129, 50.599375652333372 ], [ -1.952591194150664, 50.601011272405039 ], [ -1.949898204834256, 50.604851028019461 ], [ -1.946891000938036, 50.606197732700288 ], [ -1.944913357717927, 50.606729224185322 ], [ -1.944090020789926, 50.607458146724547 ], [ -1.946266395987304, 50.607408810621067 ], [ -1.946920312928404, 50.607702284469937 ], [ -1.947033681090084, 50.607500861228097 ], [ -1.950066280275631, 50.60753455526492 ], [ -1.950919415439656, 50.607995401690829 ], [ -1.950773633585714, 50.608231877865798 ], [ -1.951937465126802, 50.607526343214317 ], [ -1.953009984241764, 50.60756363338789 ], [ -1.954013983422958, 50.608323084507283 ], [ -1.954141435941523, 50.608106375718478 ], [ -1.955466774782322, 50.608312850604179 ], [ -1.956576667987724, 50.609283592230639 ], [ -1.957600165520036, 50.613666211660835 ], [ -1.956992577364497, 50.616560803717221 ], [ -1.953151830011443, 50.622660950181505 ], [ -1.951416803136848, 50.624543327079991 ], [ -1.947473575751904, 50.627520953524197 ], [ -1.943807139760188, 50.629167617339263 ], [ -1.933042702679926, 50.630855349252222 ], [ -1.932699155458721, 50.631847942400881 ], [ -1.931889962661278, 50.632106484386583 ], [ -1.932132773414774, 50.63237093728452 ], [ -1.930654494389322, 50.634798151647281 ], [ -1.928963316148688, 50.636711704917857 ], [ -1.927467498404883, 50.637447355875942 ], [ -1.927125189017983, 50.638436316779824 ], [ -1.926009933294985, 50.639036369099365 ], [ -1.926008981591265, 50.639643388385693 ], [ -1.925522531331181, 50.639646676923029 ], [ -1.925776392337253, 50.640064072499776 ], [ -1.925121480867839, 50.640132883126292 ], [ -1.925623015435785, 50.640481229017823 ], [ -1.925117318849626, 50.640997105314078 ], [ -1.924229295180131, 50.64091562981686 ], [ -1.924457351690884, 50.641580335254091 ], [ -1.923759457758313, 50.642033990083078 ], [ -1.925846591418261, 50.642221505082539 ], [ -1.927603730943634, 50.641752263740955 ], [ -1.929744329902695, 50.642094390166577 ], [ -1.930867868783372, 50.641667900626864 ], [ -1.932397719407371, 50.641905316909181 ], [ -1.936926956924236, 50.641186664939028 ], [ -1.939604144220253, 50.641084632237046 ], [ -1.940827907559887, 50.640689589635798 ], [ -1.944274775847964, 50.642658053035838 ], [ -1.946933488909426, 50.645164721383495 ], [ -1.947744975099738, 50.645516729660272 ], [ -1.948851077920615, 50.645446190646638 ], [ -1.949393739393455, 50.645844819251423 ], [ -1.952891200839858, 50.651700710959027 ], [ -1.952998370753706, 50.65346515845814 ], [ -1.952298010014093, 50.656419025974692 ], [ -1.951074502370556, 50.658956289739209 ], [ -1.946770403394679, 50.665472296457438 ], [ -1.942655368657151, 50.670286764374794 ], [ -1.941733459774533, 50.673051560534759 ], [ -1.942125640052261, 50.674056253053479 ], [ -1.945557482536545, 50.674140766400058 ], [ -1.946752601702865, 50.674792398168407 ], [ -1.948110940993846, 50.677596099945234 ], [ -1.948447458291448, 50.677893919767328 ], [ -1.948777390829442, 50.677725030813654 ], [ -1.948623922575219, 50.678263651812351 ], [ -1.949941413062501, 50.679787573239189 ], [ -1.949827860144961, 50.68011754094929 ], [ -1.952689964009257, 50.677189003222878 ], [ -1.953453148562329, 50.676858394911122 ], [ -1.954240010243838, 50.67691717381183 ], [ -1.954309788418534, 50.67504401458703 ], [ -1.958054159164426, 50.672336008633586 ], [ -1.959456176957401, 50.672784341546588 ], [ -1.959452877156642, 50.673335615627813 ], [ -1.960504442741074, 50.673196582519225 ], [ -1.960154521627932, 50.673673097122951 ], [ -1.959412920292684, 50.673670137859432 ], [ -1.961340873447132, 50.674898354022005 ], [ -1.961648916112551, 50.672016263374623 ], [ -1.963484191218468, 50.670475535872647 ], [ -1.964733763817753, 50.670378808451034 ], [ -1.965627630362172, 50.670985176651179 ], [ -1.966254548037925, 50.671007884133665 ], [ -1.965307515938108, 50.671440143687484 ], [ -1.965744654187674, 50.671635428445562 ], [ -1.968097549918633, 50.670326763781297 ], [ -1.968723804436534, 50.668987019916443 ], [ -1.969910899127089, 50.669160045317533 ], [ -1.970054978709475, 50.669541362587431 ], [ -1.971102192641805, 50.669413011263813 ], [ -1.968005007865897, 50.666874376645936 ], [ -1.968056693762322, 50.665824934024926 ], [ -1.969064706566179, 50.664809031184589 ], [ -1.969457429249993, 50.66361935302303 ], [ -1.968119588463473, 50.660496630841408 ], [ -1.96886925182052, 50.660677619966698 ], [ -1.968598736602087, 50.661108292541755 ], [ -1.970765454815724, 50.662304944791821 ], [ -1.9708232389574, 50.662725799364594 ], [ -1.972767744288052, 50.664253282489689 ], [ -1.972169799400355, 50.663334056182073 ], [ -1.972400687642923, 50.6628538901304 ], [ -1.974365729964991, 50.663089961872437 ], [ -1.97446742829812, 50.663442509917388 ], [ -1.975331943368305, 50.663355469777372 ], [ -1.97679425918257, 50.661745149924151 ], [ -1.976595273910719, 50.660726230237344 ], [ -1.978205876769786, 50.659263364004438 ], [ -1.976670040077135, 50.660983370261654 ], [ -1.976834091843934, 50.661216360759674 ], [ -1.977469348893246, 50.661153539283966 ], [ -1.977680727373588, 50.66297736345544 ], [ -1.976275120609127, 50.664390757641243 ], [ -1.974903932388759, 50.664770866547478 ], [ -1.974708428114731, 50.665145854113746 ], [ -1.976297276999034, 50.665469940021573 ], [ -1.978937961074082, 50.664142222549209 ], [ -1.980775985582379, 50.663856565646689 ], [ -1.981395358382741, 50.664862047777468 ], [ -1.981896264355487, 50.664725417503739 ], [ -1.982030203067329, 50.665970950899371 ], [ -1.983576394388872, 50.667076460981541 ], [ -1.983207668885875, 50.669423528925684 ], [ -1.984676594945667, 50.669237640197196 ], [ -1.98524947183026, 50.669901376646365 ], [ -1.983567593369021, 50.672279733031338 ], [ -1.983381786938373, 50.673482088460609 ], [ -1.981233309706602, 50.674290230643031 ], [ -1.981017934900219, 50.675106703416915 ], [ -1.979419955910271, 50.675536307545599 ], [ -1.979221496649749, 50.676316864819107 ], [ -1.97836639693774, 50.676945291395377 ], [ -1.979249698637147, 50.676523712116058 ], [ -1.980049047017973, 50.677227091730181 ], [ -1.981676716360727, 50.676895546350956 ], [ -1.986135445993586, 50.674495076858648 ], [ -1.988293945422492, 50.672936872674008 ], [ -1.989424749837267, 50.672557433058103 ], [ -1.991003979863841, 50.672832739782905 ], [ -1.991916769341527, 50.672705119460417 ], [ -1.991135902655055, 50.671031466836823 ], [ -1.991523690026107, 50.670671777673327 ], [ -1.992716640864064, 50.67053696880258 ], [ -1.993497922847807, 50.669406567358372 ], [ -1.993505019736473, 50.66876088597521 ], [ -1.992617925279135, 50.66803062576497 ], [ -1.993517940119669, 50.667208724372671 ], [ -1.99573233029697, 50.667154851108499 ], [ -1.997784072030744, 50.667870746954733 ], [ -1.997242093857446, 50.6688185947053 ], [ -1.997675067938991, 50.670003865430665 ], [ -1.999172203057563, 50.669277266670392 ], [ -2.001891989015379, 50.67073770580803 ], [ -2.001624515170466, 50.67126826921254 ], [ -2.002107131717445, 50.671717059200155 ], [ -2.001601917385821, 50.671962507211262 ], [ -2.001355766849425, 50.672903175286102 ], [ -1.998524065377158, 50.67287262190122 ], [ -1.997827834389586, 50.673242196636203 ], [ -1.995784293528276, 50.673380684391603 ], [ -1.996985707407088, 50.674257553037499 ], [ -1.997740029281952, 50.674273731820058 ], [ -1.997321134376391, 50.674499447668829 ], [ -1.997980632530258, 50.674942813245337 ], [ -1.99865135332141, 50.674685613474402 ], [ -1.998133436778092, 50.675150539222145 ], [ -1.99777959587948, 50.675083066695649 ], [ -1.997894252419286, 50.675701815039851 ], [ -1.999235942263757, 50.6754968198196 ], [ -1.998978300640152, 50.674878100491938 ], [ -1.999925090741883, 50.675348398779704 ], [ -1.99926985418502, 50.675853810319005 ], [ -2.000728959823514, 50.675854715352827 ], [ -1.999947744299112, 50.675947332748933 ], [ -2.000127442629356, 50.676317821122943 ], [ -1.999102859618362, 50.675943737984468 ], [ -1.998624503164618, 50.676060601012317 ], [ -1.998573550433084, 50.676632591338645 ], [ -1.999901072543802, 50.676784590466511 ], [ -1.999893969240026, 50.676469843666148 ], [ -2.001411112280077, 50.676056139866425 ], [ -2.001783354188532, 50.676207218375744 ], [ -2.003693862359347, 50.675335735363667 ], [ -2.005781415919365, 50.675580249844572 ], [ -2.007165494976172, 50.675405699867305 ], [ -2.008165951667797, 50.674770725116772 ], [ -2.008274776793407, 50.674073762788282 ], [ -2.008689448508509, 50.674001792594787 ], [ -2.008516668564413, 50.673494584900808 ], [ -2.009343135901372, 50.673463046306424 ], [ -2.009051578765702, 50.673113247004885 ], [ -2.009901921893199, 50.672511541791593 ], [ -2.009798661856489, 50.672003787220504 ], [ -2.010271281334457, 50.672703087108388 ], [ -2.010688770257861, 50.67271383483218 ], [ -2.010020946317566, 50.673146441681716 ], [ -2.009418417008417, 50.674704071105424 ], [ -2.009942107355382, 50.674955831427859 ], [ -2.010962345715467, 50.674349611870213 ], [ -2.01296626572092, 50.674200147076455 ], [ -2.012373366129031, 50.674576094750066 ], [ -2.011180333108903, 50.674541135182743 ], [ -2.009984699750569, 50.675593432344051 ], [ -2.009478039945884, 50.675538615529966 ], [ -2.009795177942634, 50.6760800084582 ], [ -2.009253314247004, 50.676900162422882 ], [ -2.007989564693327, 50.677580110894091 ], [ -2.004731514806253, 50.677458930415575 ], [ -2.00399558679415, 50.677799795377361 ], [ -2.004278670635153, 50.678125324454228 ], [ -2.005570856161931, 50.677908534139689 ], [ -2.006666442802319, 50.678683668301169 ], [ -2.008131414004316, 50.679128700961641 ], [ -2.008046552463932, 50.679620650033023 ], [ -2.008614278702004, 50.680232103995031 ], [ -2.008383721338534, 50.680891311027992 ], [ -2.009259774068723, 50.680791456835401 ], [ -2.009700082123069, 50.68127878582748 ], [ -2.011082958178855, 50.681137472276539 ], [ -2.010767335824644, 50.681262503092022 ], [ -2.011034907587475, 50.681432443672946 ], [ -2.009929432104963, 50.681400135453394 ], [ -2.009049109172691, 50.681806756705321 ], [ -2.009587016405209, 50.681859746832195 ], [ -2.009623924400657, 50.682313883198788 ], [ -2.009998902867308, 50.68204410870932 ], [ -2.009963663942548, 50.682501809204169 ], [ -2.010559603274741, 50.68256200628246 ], [ -2.01032178628726, 50.6823660227776 ], [ -2.011621015484782, 50.681782239459544 ], [ -2.010804434069459, 50.682388418554545 ], [ -2.011448568122923, 50.682770557829087 ], [ -2.012401030743741, 50.68225155086823 ], [ -2.012282239153727, 50.682615769987066 ], [ -2.014586675714445, 50.682748606721894 ], [ -2.016682798444234, 50.682285150200556 ], [ -2.016834146283692, 50.6819820808674 ], [ -2.017152707961918, 50.682175382139256 ], [ -2.018586141686914, 50.681204817381399 ], [ -2.019399038764244, 50.68212376685711 ], [ -2.019858841004949, 50.681831396155673 ], [ -2.021458554299382, 50.682278940478461 ], [ -2.022065734003343, 50.682095392587605 ], [ -2.022107806646389, 50.681263507628032 ], [ -2.021506275000787, 50.681428198107803 ], [ -2.02159192568069, 50.679957843878903 ], [ -2.021870100638703, 50.6786313265549 ], [ -2.022700461855169, 50.677748952936589 ], [ -2.021919027673367, 50.677280549755743 ], [ -2.021973885826716, 50.676585405427119 ], [ -2.021029679072726, 50.676231275750681 ], [ -2.022169126679755, 50.6765781750263 ], [ -2.02233090940621, 50.677424395868009 ], [ -2.023207071570718, 50.677599566709581 ], [ -2.024034598480697, 50.676703742173423 ], [ -2.023389052709002, 50.676442143111764 ], [ -2.023884290275955, 50.676242392622754 ], [ -2.0246090371762, 50.676426624337005 ], [ -2.026002087991392, 50.674715824156443 ], [ -2.026047122732739, 50.674280552824349 ], [ -2.02352191450028, 50.673424079156462 ], [ -2.022794011181794, 50.672437711658745 ], [ -2.023252063397897, 50.671518533775497 ], [ -2.02538942964889, 50.669903837325599 ], [ -2.026491344586498, 50.669169802676848 ], [ -2.027183225372472, 50.669013120580551 ], [ -2.026908240980082, 50.668226303693572 ], [ -2.027259027761991, 50.667963651745822 ], [ -2.027783775626279, 50.667834918711186 ], [ -2.027073793701799, 50.668229860450104 ], [ -2.027371448917844, 50.669049939026053 ], [ -2.026449019350752, 50.669394611948242 ], [ -2.025613291927658, 50.670457786271562 ], [ -2.027177257134979, 50.670886348378417 ], [ -2.026226271062652, 50.670811943862191 ], [ -2.025503350614918, 50.67120419234805 ], [ -2.024877194194224, 50.672605390998307 ], [ -2.025269463412502, 50.673012740359454 ], [ -2.027889139677531, 50.673255857034519 ], [ -2.028816742099367, 50.674345525003979 ], [ -2.033345124461631, 50.671957559224353 ], [ -2.033697008620792, 50.671324349540015 ], [ -2.034940825684079, 50.671196273546052 ], [ -2.037883211778816, 50.671604492207031 ], [ -2.041120768773589, 50.671297597566429 ], [ -2.042290523183291, 50.670691937947169 ], [ -2.044601694843387, 50.671000402739026 ], [ -2.044957880260935, 50.670505670403521 ], [ -2.044891894172582, 50.671107305581025 ], [ -2.042974563563168, 50.671322945000504 ], [ -2.043112299393155, 50.671767184404409 ], [ -2.040612206594568, 50.67230318550051 ], [ -2.03841580486166, 50.67227154876489 ], [ -2.03798579250464, 50.672499241410414 ], [ -2.037420826890417, 50.672065996860958 ], [ -2.034608878941814, 50.671977819511959 ], [ -2.03221818306627, 50.673286141629433 ], [ -2.029805464320702, 50.675688812204449 ], [ -2.028645269273865, 50.676220596771124 ], [ -2.027550483234427, 50.677276633506324 ], [ -2.027955440815616, 50.677518435251841 ], [ -2.028215705691954, 50.677349351111637 ], [ -2.027798634357251, 50.678067958787118 ], [ -2.027560606078826, 50.677567080343138 ], [ -2.026585685427151, 50.6780430751895 ], [ -2.026333225120914, 50.679583638314568 ], [ -2.027165748544469, 50.679996184257995 ], [ -2.026906188206687, 50.681505264634666 ], [ -2.028458903763818, 50.681470716547594 ], [ -2.029067318681363, 50.681121618256157 ], [ -2.029577859326653, 50.6803984285065 ], [ -2.028896912907867, 50.680244864836666 ], [ -2.029595888662981, 50.679899336882769 ], [ -2.029037857725223, 50.679256529434653 ], [ -2.029768567045204, 50.679894802758284 ], [ -2.02980720060183, 50.680614225910389 ], [ -2.028668532541301, 50.681702622063064 ], [ -2.027919742275165, 50.681673144554509 ], [ -2.027822249575891, 50.68198072791381 ], [ -2.026864139179504, 50.682214803143346 ], [ -2.024721090460262, 50.684606545890247 ], [ -2.026060037467386, 50.684455120830499 ], [ -2.028983565394981, 50.682969676428002 ], [ -2.03138086008262, 50.682202835813811 ], [ -2.033496672627149, 50.679873964270705 ], [ -2.035102528495086, 50.679091987807944 ], [ -2.037896137014513, 50.678676505767001 ], [ -2.04203046761172, 50.678800012444462 ], [ -2.046519294597523, 50.678079722155331 ], [ -2.047487595747127, 50.676912018292981 ], [ -2.047313362289868, 50.67404877944788 ], [ -2.047984695523557, 50.677275124483451 ], [ -2.051117425039451, 50.677875363357728 ], [ -2.048089650626281, 50.677492711054406 ], [ -2.046466386671781, 50.678996099510677 ], [ -2.047383737950782, 50.679190865720415 ], [ -2.051368188992765, 50.678062321922638 ], [ -2.052671561240908, 50.677978983712784 ], [ -2.048621910137885, 50.678932243034254 ], [ -2.047695410822421, 50.679497353723761 ], [ -2.046356394078743, 50.679385537321316 ], [ -2.045466104734921, 50.680818472658267 ], [ -2.045966182269977, 50.681260721837582 ], [ -2.04637624954556, 50.680890048131637 ], [ -2.046388116942225, 50.681436801177185 ], [ -2.045032116868968, 50.681403135990017 ], [ -2.045092139592714, 50.680491318467737 ], [ -2.044701276538118, 50.680293598495766 ], [ -2.043155887909287, 50.680542417178891 ], [ -2.041926967169009, 50.680095900281344 ], [ -2.040401300688218, 50.680294337580804 ], [ -2.040088097910205, 50.679795312639698 ], [ -2.036898352188198, 50.680502382670412 ], [ -2.034403475993828, 50.683023895845395 ], [ -2.032000410966697, 50.683623537819365 ], [ -2.030313998923355, 50.684953160671363 ], [ -2.03031849588413, 50.685216673885883 ], [ -2.032012757972951, 50.685138792785445 ], [ -2.027241687915206, 50.68631006355362 ], [ -2.027056462467701, 50.686730941348074 ], [ -2.027522148598635, 50.68654921815186 ], [ -2.027595898665624, 50.686879237289546 ], [ -2.027182743231756, 50.687121258443149 ], [ -2.026957504838569, 50.686877589565363 ], [ -2.025810883134844, 50.686966875945608 ], [ -2.024456509769527, 50.687636243757503 ], [ -2.024536271800466, 50.688438415254687 ], [ -2.025014865868897, 50.688644288069383 ], [ -2.026285928542728, 50.688391272767952 ], [ -2.025983308543501, 50.688866135287277 ], [ -2.026419548646323, 50.689349874111961 ], [ -2.026803284193988, 50.689437890021907 ], [ -2.027863297796247, 50.688871978421837 ], [ -2.027287357045525, 50.689306508184806 ], [ -2.027764720122146, 50.689715530379054 ], [ -2.026986017076279, 50.689588924970174 ], [ -2.026250295975472, 50.690366088129544 ], [ -2.025624139068364, 50.689593747407677 ], [ -2.024610534303368, 50.689634448027832 ], [ -2.023996680906464, 50.690673265250346 ], [ -2.024828579623517, 50.692276500409172 ], [ -2.026404261225345, 50.692171825619475 ], [ -2.025729174469138, 50.69257486115081 ], [ -2.02574231156427, 50.693280789607208 ], [ -2.02451121503215, 50.694451941784806 ], [ -2.024520363807582, 50.695650692478701 ], [ -2.025461046440911, 50.696646866876989 ], [ -2.02605662255407, 50.698216872217358 ], [ -2.026477476671125, 50.698790519806053 ], [ -2.027906461653583, 50.699220001377512 ], [ -2.028907541172081, 50.699208958938243 ], [ -2.027928489947767, 50.699040380432855 ], [ -2.027813998685866, 50.698442181721838 ], [ -2.027374942379719, 50.698305548674384 ], [ -2.028085374835607, 50.697766765793382 ], [ -2.028680051581526, 50.697757619521774 ], [ -2.029175196579422, 50.697060502544332 ], [ -2.030282429513344, 50.697055706235872 ], [ -2.029345239631516, 50.697267292529894 ], [ -2.029216862716457, 50.698011923038564 ], [ -2.028696033353855, 50.698352895556695 ], [ -2.030297770437966, 50.698833581755459 ], [ -2.031876237673483, 50.698397881703087 ], [ -2.031206980773658, 50.699120198995779 ], [ -2.029636706605596, 50.699113444397049 ], [ -2.029742096116123, 50.700120598715444 ], [ -2.029179234907788, 50.701181896895065 ], [ -2.031804351247501, 50.70094289230758 ], [ -2.032578580983227, 50.700367114754627 ], [ -2.031993184452521, 50.701588489863774 ], [ -2.033202831906186, 50.702114173640794 ], [ -2.031288165828102, 50.7018386944683 ], [ -2.031010778333778, 50.702126502406301 ], [ -2.031552036541227, 50.702565232285842 ], [ -2.02973572241507, 50.703303145900804 ], [ -2.02803086461139, 50.703486170819168 ], [ -2.025015655084882, 50.702851111649338 ], [ -2.024399351432495, 50.702198407622419 ], [ -2.024842293455384, 50.701725240364986 ], [ -2.023293190300278, 50.701887473812945 ], [ -2.023754653044297, 50.701560930463735 ], [ -2.022004013822397, 50.700745659770234 ], [ -2.018965191600694, 50.70059607382894 ], [ -2.018083421633367, 50.701574646439269 ], [ -2.019332847797518, 50.70265894636006 ], [ -2.022307178377853, 50.703735732999974 ], [ -2.023686919980364, 50.704704824762906 ], [ -2.024839149917422, 50.706393411452751 ], [ -2.026248871145153, 50.7074353394204 ], [ -2.027452413848091, 50.707068128610814 ], [ -2.029016102450431, 50.707352800754016 ], [ -2.031041242678915, 50.707268613974655 ], [ -2.032459196387305, 50.707811401058862 ], [ -2.034195602182685, 50.707884593241488 ], [ -2.037752601843952, 50.707191020737447 ], [ -2.039875405445803, 50.707157874564174 ], [ -2.041586944247707, 50.706388367774586 ], [ -2.041866870684254, 50.704364884763073 ], [ -2.043018331315301, 50.703023627871232 ], [ -2.044306793811436, 50.702803735479229 ], [ -2.04616514840753, 50.701929758215783 ], [ -2.047665933317038, 50.701702509877364 ], [ -2.048751231261793, 50.700940381438599 ], [ -2.053478452437961, 50.700240385869364 ], [ -2.05452015138048, 50.69980283449032 ], [ -2.05550050968054, 50.699052396784076 ], [ -2.056943869774831, 50.696046247279725 ], [ -2.05932450367905, 50.693190718969724 ], [ -2.059280232888584, 50.690664648822505 ], [ -2.062677310180209, 50.689181711768001 ], [ -2.063246257307862, 50.688016871655691 ], [ -2.065723119757321, 50.686720468455867 ], [ -2.068085887366083, 50.68684948677182 ], [ -2.068893565847648, 50.686467741503549 ], [ -2.07156708221978, 50.686145071089754 ], [ -2.072534834000447, 50.686736200983248 ], [ -2.072532923020604, 50.687331494569833 ], [ -2.072874859469886, 50.686915805418984 ], [ -2.073217893433236, 50.687218646936167 ], [ -2.07324283365983, 50.68687780253812 ], [ -2.075821771247028, 50.687619846834743 ], [ -2.077966242980394, 50.686677770187842 ], [ -2.078047400541839, 50.686968185821172 ], [ -2.079083082073766, 50.68666801728542 ], [ -2.081168700919199, 50.686957041483794 ], [ -2.081474173850393, 50.686789559204485 ], [ -2.081197234675292, 50.686280754256096 ], [ -2.082139804217372, 50.686972547002668 ], [ -2.081681179274033, 50.687785826231391 ], [ -2.080683212335555, 50.687814400646431 ], [ -2.083228861043667, 50.690426819292966 ], [ -2.084238453207311, 50.690577190712865 ], [ -2.086080709465507, 50.689238628204016 ], [ -2.089091529963977, 50.689096038186833 ], [ -2.090686004691535, 50.688595708553336 ], [ -2.090676085545582, 50.687864592326768 ], [ -2.088007432168844, 50.686999690244022 ], [ -2.087169479932343, 50.686277323346978 ], [ -2.087656305256341, 50.684677122873211 ], [ -2.088159729948455, 50.684426748023888 ], [ -2.089040307197525, 50.684494431900369 ], [ -2.089343940473647, 50.68412188119612 ], [ -2.089367094046748, 50.682904241393473 ], [ -2.089931774404296, 50.683613380722754 ], [ -2.08931229168971, 50.684594957133761 ], [ -2.090665134370715, 50.685127176450528 ], [ -2.092623264048048, 50.685346869231211 ], [ -2.095283110644643, 50.684667562572507 ], [ -2.095617170746984, 50.683973943040563 ], [ -2.095123693910227, 50.681444671039067 ], [ -2.097023088442137, 50.680703885570175 ], [ -2.098132172023207, 50.679740722339076 ], [ -2.099266996286639, 50.679614773895558 ], [ -2.102175464152179, 50.682138350584133 ], [ -2.103644917435451, 50.68288075351667 ], [ -2.110226132495087, 50.683769542160505 ], [ -2.112247422068137, 50.682558982887237 ], [ -2.11204188465942, 50.681842451529484 ], [ -2.111120618752234, 50.681359561450961 ], [ -2.112258456984444, 50.681840443581422 ], [ -2.113668275033693, 50.681863364066068 ], [ -2.113933177939959, 50.680783965951754 ], [ -2.115225078131756, 50.680632517351974 ], [ -2.115690546032259, 50.678838840343467 ], [ -2.115015842847321, 50.678416885467172 ], [ -2.115227096971011, 50.678006623943581 ], [ -2.115594724440756, 50.677844367298079 ], [ -2.117283825456505, 50.678093586604795 ], [ -2.11672393735604, 50.677187670064463 ], [ -2.117525278176302, 50.677864934624587 ], [ -2.117275766057136, 50.678268059737796 ], [ -2.116022472859174, 50.677977038104331 ], [ -2.115487684216578, 50.678061200908161 ], [ -2.115274259245253, 50.678176519106167 ], [ -2.115928416775495, 50.678812722505143 ], [ -2.115692581433187, 50.67945110346276 ], [ -2.115481385730277, 50.680674502264509 ], [ -2.114931551169595, 50.680986197342733 ], [ -2.113997257865851, 50.680948453854377 ], [ -2.114047596658136, 50.681878269296099 ], [ -2.112485550228506, 50.68209024381671 ], [ -2.112557630552996, 50.68265674113227 ], [ -2.111499846360234, 50.683642432062975 ], [ -2.112491565084856, 50.683438246358619 ], [ -2.109970884554996, 50.684172624145518 ], [ -2.108139510608094, 50.683688758232257 ], [ -2.103824146550954, 50.683272698008793 ], [ -2.102038976260647, 50.682523349934108 ], [ -2.099343366459189, 50.680240607864519 ], [ -2.098645241317625, 50.680066702448464 ], [ -2.097322422367697, 50.681034574672097 ], [ -2.095680143609572, 50.681559303999357 ], [ -2.096083210796178, 50.684137231844439 ], [ -2.095546815977634, 50.68487598342351 ], [ -2.092359184737961, 50.685676207024393 ], [ -2.088101173285569, 50.684906113798313 ], [ -2.08777142383547, 50.686452235547485 ], [ -2.09081264721375, 50.687467905652319 ], [ -2.091115058609162, 50.68865382918397 ], [ -2.089410475289294, 50.689318828202204 ], [ -2.086255181865683, 50.689441695043058 ], [ -2.085263482833906, 50.690616899895851 ], [ -2.084507972922131, 50.690849456189376 ], [ -2.082885398212193, 50.690708527315174 ], [ -2.08014181629729, 50.688259889080172 ], [ -2.079201800307886, 50.688206595304855 ], [ -2.078415515371637, 50.688717061089847 ], [ -2.078682425841224, 50.693359877575098 ], [ -2.079389213303639, 50.694379196778875 ], [ -2.080266545005633, 50.694121381735293 ], [ -2.080874241651504, 50.693480666810906 ], [ -2.082872417534834, 50.692994537706404 ], [ -2.083852201264435, 50.693041528582768 ], [ -2.08479434433374, 50.693438303293512 ], [ -2.084410675549821, 50.693405297937986 ], [ -2.08461138224166, 50.694013970361731 ], [ -2.085093918274484, 50.693876024802655 ], [ -2.085182527187503, 50.694361592340051 ], [ -2.084665533725447, 50.694209075706503 ], [ -2.08489250855492, 50.694456205257239 ], [ -2.086871286892551, 50.694906237006094 ], [ -2.087544313343034, 50.695929099631435 ], [ -2.089358493837177, 50.69620740884374 ], [ -2.091111775343911, 50.697154783090951 ], [ -2.092197202761529, 50.696841891119163 ], [ -2.093413076039904, 50.695972191108339 ], [ -2.094993460910567, 50.696827060833975 ], [ -2.099972445973479, 50.695160997360922 ], [ -2.101522103990896, 50.695497745946149 ], [ -2.100186458159945, 50.695267824868587 ], [ -2.099253777494817, 50.695438575808204 ], [ -2.096730274505525, 50.696580100189848 ], [ -2.094995223858581, 50.697005965576487 ], [ -2.093447282371956, 50.696091737006284 ], [ -2.092432271711962, 50.696880356292304 ], [ -2.090877180669772, 50.697364452574753 ], [ -2.089336109648419, 50.696344977857137 ], [ -2.087357491814284, 50.69597509780872 ], [ -2.086480721340193, 50.695040489131891 ], [ -2.08471159456089, 50.694604762893633 ], [ -2.084179050912451, 50.693694166256897 ], [ -2.083216955390359, 50.693300908034708 ], [ -2.081694957595001, 50.694060138013711 ], [ -2.081648203374104, 50.694849737458505 ], [ -2.080258718308618, 50.69615736761245 ], [ -2.079669981958063, 50.696307060729332 ], [ -2.079354674446388, 50.695721842889633 ], [ -2.078760263683528, 50.695863434556323 ], [ -2.077906060971331, 50.695584336662662 ], [ -2.077555166665723, 50.694895675973115 ], [ -2.076908704547813, 50.695213599847357 ], [ -2.076614266588088, 50.696097765968695 ], [ -2.075742854489966, 50.696570493112851 ], [ -2.075721756426217, 50.697531840962334 ], [ -2.073615767990069, 50.698069176002093 ], [ -2.072360450537519, 50.697531296621058 ], [ -2.071054178837491, 50.697917903695831 ], [ -2.070795420351279, 50.698181557626086 ], [ -2.071454048426525, 50.698301651648123 ], [ -2.072515346075245, 50.699729059858782 ], [ -2.072487518163784, 50.700946703191768 ], [ -2.073070981359455, 50.701928364586458 ], [ -2.07244516656765, 50.702890088042977 ], [ -2.07240606655312, 50.704108645752875 ], [ -2.072784643106501, 50.704434844131121 ], [ -2.072162564982244, 50.705044062202312 ], [ -2.070777392594569, 50.704927957093432 ], [ -2.069694607592025, 50.704326987021766 ], [ -2.067811003128909, 50.704203153917334 ], [ -2.066601583739563, 50.705143603181099 ], [ -2.064554234242469, 50.705410066031916 ], [ -2.063369148728029, 50.70656004504513 ], [ -2.0611651373131, 50.706211336441193 ], [ -2.059798647500193, 50.705194088703422 ], [ -2.058971547962798, 50.705133385369486 ], [ -2.057632463630654, 50.705619714703168 ], [ -2.060980096690301, 50.706589205255831 ], [ -2.063194974418573, 50.707636548937558 ], [ -2.064130850397821, 50.707486749758608 ], [ -2.064821145410349, 50.707930606384807 ], [ -2.066220401534621, 50.707964880907525 ], [ -2.067334332464466, 50.707548790870781 ], [ -2.067897352419665, 50.708080813780917 ], [ -2.069200785936345, 50.708437052935125 ], [ -2.070663420349902, 50.708230233403754 ], [ -2.071345142246951, 50.707647079397368 ], [ -2.071900656270711, 50.707914726490451 ], [ -2.071907312082458, 50.708635656930852 ], [ -2.070983236540488, 50.708998889984038 ], [ -2.070858575095626, 50.709913568532102 ], [ -2.070353206876469, 50.710042504782706 ], [ -2.070155289089804, 50.70936633418043 ], [ -2.069723625360223, 50.709556348929993 ], [ -2.068981083578069, 50.711190771621055 ], [ -2.068328561990453, 50.711444768238479 ], [ -2.066471952817877, 50.713565471814839 ], [ -2.065000022631691, 50.714343285928223 ], [ -2.062902490924976, 50.71654046332629 ], [ -2.057797025057317, 50.71723022151874 ], [ -2.055603602297086, 50.717802311231722 ], [ -2.051239059320826, 50.717527349876207 ], [ -2.048371040006492, 50.716544806569658 ], [ -2.044995345821145, 50.714858278276687 ], [ -2.041626307616742, 50.714031304134785 ], [ -2.046065830885373, 50.715933385824187 ], [ -2.046942755145102, 50.71751122668153 ], [ -2.046243594082323, 50.716642850770107 ], [ -2.045661473613156, 50.71671863272627 ], [ -2.045569345283063, 50.718058579379708 ], [ -2.046246569101706, 50.718197699828913 ], [ -2.045204063321741, 50.718255677085324 ], [ -2.043830858256668, 50.717643820613254 ], [ -2.044667433462762, 50.718483451013007 ], [ -2.044517595894884, 50.718835982848439 ], [ -2.040737838266395, 50.719997449676981 ], [ -2.040964772449268, 50.72025998062901 ], [ -2.04414728140624, 50.719685042696973 ], [ -2.041335778996081, 50.72332460254465 ], [ -2.042202124581709, 50.724104850994181 ], [ -2.042140096818789, 50.724472716014148 ], [ -2.042405213324492, 50.724685685114551 ], [ -2.041569665373244, 50.725060077441128 ], [ -2.042242362896926, 50.726280189735576 ], [ -2.043025935886486, 50.72644627547443 ], [ -2.045931459310038, 50.726127686214546 ], [ -2.046184586597117, 50.725649160711995 ], [ -2.047417067330802, 50.725538943230958 ], [ -2.047963435983713, 50.725103461104453 ], [ -2.047983234996419, 50.723710462154777 ], [ -2.047508314706962, 50.723374338130185 ], [ -2.046794300742476, 50.72337822427717 ], [ -2.045767268996466, 50.723424514043273 ], [ -2.045711512765635, 50.722922760067682 ], [ -2.046202916834824, 50.72272919267548 ], [ -2.048148038026258, 50.722797617969697 ], [ -2.048965951453484, 50.723294596671714 ], [ -2.049972078017222, 50.724893071116355 ], [ -2.050886166913397, 50.725155257988888 ], [ -2.050735430205624, 50.725908911131299 ], [ -2.051922213190823, 50.726777087389344 ], [ -2.050952472505483, 50.726166912211916 ], [ -2.05042121877307, 50.726195923644781 ], [ -2.051026220572021, 50.727527488172349 ], [ -2.051729646547254, 50.72815487202778 ], [ -2.053348311256592, 50.728756632756358 ], [ -2.051444877433447, 50.728124402584562 ], [ -2.050057125655732, 50.72620327121701 ], [ -2.049506216448317, 50.726404081944835 ], [ -2.049799751167407, 50.726648534773943 ], [ -2.04828568730122, 50.7271267005532 ], [ -2.048053417813319, 50.728544068625212 ], [ -2.046298174728145, 50.730170692026299 ], [ -2.044315288681805, 50.730934061074947 ], [ -2.04339782420691, 50.730161040645385 ], [ -2.043609701790764, 50.730974808323076 ], [ -2.042647903004397, 50.731272833089328 ], [ -2.042570548856495, 50.731900560832386 ], [ -2.041493165704944, 50.731737773296139 ], [ -2.042007655243443, 50.731442157213166 ], [ -2.04117051054682, 50.730183451844177 ], [ -2.039741796975759, 50.729615616389218 ], [ -2.037278496494612, 50.730352949751413 ], [ -2.036113341579705, 50.731567355924646 ], [ -2.034787431892901, 50.730130766103834 ], [ -2.034897439222671, 50.729454406553636 ], [ -2.036003016899991, 50.728244608231741 ], [ -2.035188169312437, 50.727879682681007 ], [ -2.0339273621706, 50.728202911443979 ], [ -2.032511251841012, 50.729150275691147 ], [ -2.030178035755218, 50.729767847889839 ], [ -2.027518517417076, 50.729697535374299 ], [ -2.02686934975375, 50.729326303264934 ], [ -2.025792715407628, 50.729523538477132 ], [ -2.024773818080365, 50.729359165085157 ], [ -2.024548713216975, 50.729595748405593 ], [ -2.022189379630622, 50.734849848359183 ], [ -2.022521812365452, 50.736565549690162 ], [ -2.016615231188705, 50.736334570267211 ], [ -2.016239835606585, 50.736695217559678 ], [ -2.013528987623319, 50.737053548128124 ], [ -2.012737216664316, 50.748300656086172 ], [ -2.012706069770054, 50.748501226770088 ], [ -2.010256596974578, 50.77378808669306 ], [ -2.009320668864138, 50.782589239754763 ], [ -2.007960346652959, 50.782420239981889 ], [ -2.006075197402891, 50.78276835907112 ], [ -2.001198404068241, 50.784808983685423 ], [ -2.000466461162524, 50.785347587680391 ], [ -1.999132861837501, 50.788128124335067 ], [ -1.999565474806699, 50.78960473875452 ], [ -2.004867277522623, 50.790202647528439 ], [ -2.005874608459803, 50.79092647885556 ], [ -1.997166216833597, 50.792839325929933 ], [ -1.993991101416281, 50.79236175159572 ], [ -1.991568036355622, 50.791070284363265 ], [ -1.991095498654218, 50.791405666737518 ], [ -1.988238377566523, 50.790885621020173 ], [ -1.98483174340313, 50.791456280220302 ], [ -1.983624625868571, 50.790942622141586 ], [ -1.981429476829827, 50.791762470202329 ], [ -1.980914603048132, 50.791629250837708 ], [ -1.980567234148495, 50.790982638604866 ], [ -1.979855105797868, 50.790945666366788 ], [ -1.978363265155736, 50.792507424422453 ], [ -1.969666159124753, 50.7924633285581 ], [ -1.968792298432271, 50.792339945243597 ], [ -1.966293538289845, 50.790868047269221 ], [ -1.963853357021078, 50.790807095094905 ], [ -1.958327261080469, 50.789293586226535 ], [ -1.955192353826055, 50.790337362267614 ], [ -1.952309349442747, 50.790504439173255 ], [ -1.950356816278251, 50.790820170526459 ], [ -1.945414884314715, 50.794847600815253 ], [ -1.942404955815493, 50.796452241179963 ], [ -1.941730199201304, 50.797076003256862 ], [ -1.941613886203911, 50.798220659173339 ], [ -1.940880877657392, 50.79880752877245 ], [ -1.939687522458299, 50.798807822161983 ], [ -1.937991102020086, 50.798352845777977 ], [ -1.936354813261168, 50.797404135241599 ], [ -1.935239583436567, 50.79429920739922 ], [ -1.936329412269297, 50.788945492959357 ], [ -1.937439285848802, 50.78750540792177 ], [ -1.939023825504058, 50.786535036674529 ], [ -1.938058876423019, 50.785632530025339 ], [ -1.936972745330692, 50.785289323939587 ], [ -1.937501252465131, 50.783565725966596 ], [ -1.937124991184261, 50.782787592007857 ], [ -1.935052183223302, 50.782023886649561 ], [ -1.932588466790979, 50.779990919586105 ], [ -1.926998766863221, 50.780797776532268 ], [ -1.925511015275969, 50.781563871822073 ], [ -1.92505685453005, 50.782621199113343 ], [ -1.92423452440618, 50.783296012080946 ], [ -1.922403160178277, 50.78334692963071 ], [ -1.920925422254373, 50.78398261574749 ], [ -1.919505535831244, 50.784002306104199 ], [ -1.919138860893687, 50.783531762018399 ], [ -1.919699538436368, 50.782499703099845 ], [ -1.918611940076577, 50.78061761088572 ], [ -1.918893482766902, 50.779357000794548 ], [ -1.920915794413862, 50.777835990188358 ], [ -1.921595087374345, 50.776225744783559 ], [ -1.920738089333437, 50.775567776915231 ], [ -1.91934436018628, 50.775332148927511 ], [ -1.919223486328572, 50.774685383774127 ], [ -1.920027262045548, 50.774007895090769 ], [ -1.919596743430231, 50.773641592986742 ], [ -1.918687832495473, 50.773568124947076 ], [ -1.917615359296824, 50.774565558669394 ], [ -1.916982736153585, 50.774631627986899 ], [ -1.914978065378061, 50.773397205164763 ], [ -1.914223452728274, 50.773420015141525 ], [ -1.913144674885026, 50.773971387514734 ], [ -1.912955202090195, 50.775211376023314 ], [ -1.912473748461402, 50.775541043805489 ], [ -1.911354781433979, 50.775529390246433 ], [ -1.910789655360426, 50.775133282884319 ], [ -1.90927312756189, 50.775296608647047 ], [ -1.907621092389026, 50.776646982754933 ], [ -1.905985763739423, 50.777366851810228 ], [ -1.901935699435357, 50.777689881474053 ], [ -1.900848579180986, 50.77871682755044 ], [ -1.898603954229625, 50.778472202563762 ], [ -1.898074381181611, 50.778722581589477 ], [ -1.898620195381598, 50.778807645804079 ], [ -1.89839247011011, 50.77921030939946 ], [ -1.897683061110224, 50.779282523848956 ], [ -1.89749505654362, 50.779644763213305 ], [ -1.897129371474823, 50.779538362467491 ], [ -1.89621419846542, 50.780331685217512 ], [ -1.893892911741998, 50.780091306964046 ], [ -1.89283105082079, 50.779264774686922 ], [ -1.890186923296943, 50.778180559977741 ], [ -1.886730559387275, 50.775774389555195 ], [ -1.886701849269252, 50.775359818727949 ], [ -1.887822164030787, 50.77477183315537 ], [ -1.887357100145135, 50.773553746039589 ], [ -1.885553122352497, 50.772397310013538 ], [ -1.8835118002324, 50.772084176851891 ], [ -1.882061906234025, 50.771770696623257 ], [ -1.880940228836776, 50.770620348622927 ], [ -1.877838286997392, 50.769731391343662 ], [ -1.874207896154303, 50.765580103581193 ], [ -1.874891668898177, 50.764347889911036 ], [ -1.874746900713343, 50.763876464419532 ], [ -1.873382793858718, 50.763380463126971 ], [ -1.873446850572291, 50.763295033005008 ], [ -1.871369889597073, 50.763755088630845 ], [ -1.869807944818145, 50.76303218396869 ], [ -1.868714929340289, 50.76298594124502 ], [ -1.864750239383989, 50.764126323881449 ], [ -1.859092432619426, 50.768600871400785 ], [ -1.85725196573731, 50.769486273710157 ], [ -1.856849005002334, 50.770046957337023 ], [ -1.856950012952551, 50.771324948740308 ], [ -1.855981510267295, 50.7727167161393 ], [ -1.855162762182736, 50.772880324897862 ], [ -1.854182451047552, 50.772540098419391 ], [ -1.853267406722445, 50.772648645238938 ], [ -1.852316377107848, 50.773382226286813 ], [ -1.851348551049092, 50.773610259414077 ], [ -1.847919785807795, 50.771267740154073 ], [ -1.847994984043534, 50.769957623307015 ], [ -1.847249805725561, 50.769286705880752 ], [ -1.843758200310937, 50.768570692206829 ], [ -1.84508917742917, 50.767859380306056 ], [ -1.845316354155742, 50.767321898767818 ], [ -1.845075665586643, 50.766776684434696 ], [ -1.844030921697268, 50.766287846658223 ], [ -1.842773774265731, 50.764395888967698 ], [ -1.843610147685764, 50.762727976676913 ], [ -1.842132742432424, 50.761472412331152 ], [ -1.839709017854386, 50.760842346652531 ], [ -1.8377630302023, 50.759861290523503 ], [ -1.836778656214996, 50.759155739238587 ], [ -1.832455331544814, 50.757753979361716 ], [ -1.824144716726502, 50.757409871202817 ], [ -1.821198344369045, 50.757522247654535 ], [ -1.819273605587505, 50.758142459723302 ], [ -1.818746622432626, 50.757667774020234 ], [ -1.817138205372866, 50.75712925085589 ], [ -1.81617795255706, 50.756144842121586 ], [ -1.815750012386118, 50.755374345856964 ], [ -1.814331519985059, 50.754511546536065 ], [ -1.816947116062202, 50.7537603094883 ], [ -1.817303342681988, 50.753285215310683 ], [ -1.816514029879633, 50.750326246024834 ], [ -1.815010284913727, 50.748466799584627 ], [ -1.815271405844296, 50.746574199429674 ], [ -1.814842027547837, 50.744376539915287 ], [ -1.814993753763821, 50.744006285810215 ], [ -1.815438582323498, 50.744045645443897 ], [ -1.815560270349757, 50.74337677111717 ], [ -1.811113583001315, 50.74214306995642 ], [ -1.810741892252089, 50.741528238024998 ], [ -1.80948055843923, 50.741546881627187 ], [ -1.808000265388038, 50.742079595636753 ], [ -1.80738550942329, 50.74165234797055 ], [ -1.806345466246921, 50.741264820500248 ], [ -1.803769672167436, 50.741113057928388 ], [ -1.802871407066075, 50.740039573918359 ], [ -1.801470575217678, 50.739530907623376 ], [ -1.798468071961058, 50.739103076287172 ], [ -1.796317757029859, 50.739553487880066 ], [ -1.794807921779785, 50.737407811882058 ], [ -1.793822685737133, 50.737166867030268 ], [ -1.793974383191925, 50.737478288670076 ], [ -1.792955662967136, 50.73712036268369 ], [ -1.792870603747855, 50.736486183170669 ], [ -1.793179023456034, 50.734984047092915 ], [ -1.792758745508899, 50.734520153281466 ], [ -1.791957385046222, 50.734384660844114 ], [ -1.791245645127532, 50.733205434458981 ], [ -1.791309711157666, 50.732495107874563 ], [ -1.79195077205972, 50.73171925289062 ], [ -1.793934385453242, 50.731729044479366 ], [ -1.794213336503228, 50.731443566824893 ], [ -1.79362500393477, 50.730861556493181 ], [ -1.792430958765584, 50.730468297131253 ], [ -1.789192558782783, 50.730031793154794 ], [ -1.789953596220255, 50.730625781797549 ], [ -1.786832465926179, 50.729641739518812 ], [ -1.785362239864107, 50.730163345680793 ], [ -1.786954685268841, 50.729871279005124 ], [ -1.786647389496117, 50.730136889516956 ], [ -1.784461664290122, 50.730355040485293 ], [ -1.779768589165971, 50.729852571562382 ], [ -1.776148687582359, 50.730708141256819 ], [ -1.774949937305662, 50.730426163243777 ], [ -1.774605854960094, 50.730970472715114 ], [ -1.773881251730086, 50.730208283376385 ], [ -1.772246518746397, 50.730437121308441 ], [ -1.77056819447297, 50.730298962218683 ], [ -1.769959558733128, 50.730752801448347 ], [ -1.773000109984537, 50.733102275933184 ], [ -1.774086853566182, 50.73458000421337 ], [ -1.773874196957855, 50.735495951833215 ], [ -1.773269984822054, 50.73590393627741 ], [ -1.774762055014295, 50.737982279372829 ], [ -1.777191380777104, 50.739088554984775 ], [ -1.778193560656722, 50.738419660408049 ], [ -1.778502685834142, 50.738689994363412 ], [ -1.778207912310702, 50.738992436933692 ], [ -1.776846226254133, 50.739261406615164 ], [ -1.774195281798593, 50.737971273062136 ], [ -1.773463620334571, 50.738353830674534 ], [ -1.773911043832998, 50.738726992709999 ], [ -1.77313649121535, 50.7385986765333 ], [ -1.771147641041779, 50.739310575514928 ], [ -1.77262505725716, 50.740880701327335 ], [ -1.77145633163023, 50.739940635873907 ], [ -1.770984866610875, 50.739269778569117 ], [ -1.773208031016863, 50.738470250870627 ], [ -1.774053072055567, 50.737783100382622 ], [ -1.773049945288895, 50.736277594500621 ], [ -1.772544499111476, 50.736469895776871 ], [ -1.772536787562125, 50.736902460182762 ], [ -1.771588274919534, 50.73671534308464 ], [ -1.769341615458137, 50.731367588657811 ], [ -1.76938499039487, 50.730329008235557 ], [ -1.771774602886722, 50.729589133604286 ], [ -1.770719041156106, 50.7290033986338 ], [ -1.769168993524669, 50.728724286645303 ], [ -1.767130755861125, 50.728889272766544 ], [ -1.766150692050837, 50.728529427121018 ], [ -1.765614901375089, 50.728864663570043 ], [ -1.765675201999322, 50.727293759248532 ], [ -1.76627214950124, 50.726047687349052 ], [ -1.766846491760663, 50.725934624874789 ], [ -1.766010388204715, 50.724556171103721 ], [ -1.764333481221467, 50.723328894187205 ], [ -1.760291373220975, 50.722834166251602 ], [ -1.760346266164288, 50.723175988815761 ], [ -1.761959469704872, 50.724101052900153 ], [ -1.761995979261169, 50.725288176446121 ], [ -1.760060813420264, 50.723599884708968 ], [ -1.757523703933804, 50.723567668073727 ], [ -1.758874573660936, 50.724237760613413 ], [ -1.759428827590662, 50.725837802347137 ], [ -1.75892985490217, 50.725898797666403 ], [ -1.759494457476959, 50.726866672298087 ], [ -1.758293586294776, 50.725654684259595 ], [ -1.757448914700559, 50.726533317542334 ], [ -1.756042859984689, 50.725829895265285 ], [ -1.756155788198632, 50.727004501273342 ], [ -1.757346298569753, 50.727753409703496 ], [ -1.757206368189659, 50.728236026215093 ], [ -1.757882263373499, 50.728488305718137 ], [ -1.757543538507477, 50.728792465723615 ], [ -1.758130048731538, 50.729077848554645 ], [ -1.757731229634643, 50.729210978937544 ], [ -1.757976685069312, 50.729412056482325 ], [ -1.757245705201874, 50.729397956592273 ], [ -1.757087233922529, 50.730179990636756 ], [ -1.755454520714206, 50.729458947750352 ], [ -1.754167386641812, 50.727407747009359 ], [ -1.75303978766273, 50.726563653066165 ], [ -1.75042493153365, 50.725933118434952 ], [ -1.747525879365998, 50.726529401813302 ], [ -1.747436507546549, 50.72681247447759 ], [ -1.74732431294766, 50.726599102262277 ], [ -1.74586162138628, 50.726976295719581 ], [ -1.745373539087057, 50.727362857413297 ], [ -1.745780270910865, 50.727603843754444 ], [ -1.745762393998765, 50.727244091971102 ], [ -1.746724807280484, 50.72716705234081 ], [ -1.746567730699781, 50.727657709913757 ], [ -1.745499537994971, 50.727897265884792 ], [ -1.743718758509794, 50.727620889916118 ], [ -1.74342764381778, 50.728260575504848 ], [ -1.743094512564279, 50.727510750838057 ], [ -1.741470470440911, 50.727064731060139 ], [ -1.740564794918269, 50.727126576621302 ], [ -1.740794226123247, 50.726369940306647 ], [ -1.741395249674197, 50.72605558434541 ], [ -1.740681521307144, 50.725480290712611 ], [ -1.741475898110124, 50.725043177651358 ], [ -1.742007223686962, 50.72529255096795 ], [ -1.742312916681304, 50.723532397314578 ], [ -1.741127061550023, 50.724070268161604 ], [ -1.73694902812239, 50.726881977762957 ], [ -1.73546396085322, 50.727194286018566 ], [ -1.734890342047523, 50.72815879754188 ], [ -1.734006531068952, 50.728619040880325 ], [ -1.734184712690612, 50.728930588916427 ], [ -1.730189261301538, 50.731888171303211 ], [ -1.724585161339232, 50.734088230464664 ], [ -1.72221545989486, 50.734655465642653 ], [ -1.715592109171545, 50.735852679541757 ], [ -1.707328170584478, 50.736706374327277 ], [ -1.698441213433152, 50.736773739276686 ], [ -1.697959610952892, 50.73674371574063 ], [ -1.697897266803945, 50.736301970747242 ], [ -1.697766758478544, 50.736757611300789 ], [ -1.695851586122748, 50.736874019430182 ], [ -1.694384477947991, 50.736721848877487 ], [ -1.694237627193676, 50.736198060530548 ], [ -1.693979597392615, 50.736870932849271 ], [ -1.692375707753045, 50.736830778160993 ], [ -1.692547967083003, 50.737352830058569 ], [ -1.691977631067641, 50.739412429234889 ], [ -1.690820510354899, 50.740603638221749 ], [ -1.689018492228458, 50.741572772741783 ], [ -1.688575971972462, 50.74141426628556 ], [ -1.687917997885636, 50.742117526187684 ], [ -1.684645431682293, 50.743820920119376 ], [ -1.682805364506658, 50.74661088418641 ], [ -1.681674425793193, 50.750151000018192 ], [ -1.681818801760133, 50.75185004570897 ], [ -1.690613383424385, 50.751606471681413 ], [ -1.703920665229782, 50.752561545757715 ], [ -1.712704216212478, 50.752495152391774 ], [ -1.712700111592905, 50.752249633832378 ], [ -1.713593200394353, 50.752244611893538 ], [ -1.714985136589757, 50.752025845682475 ], [ -1.717969530988448, 50.752169745699774 ], [ -1.728134939433707, 50.750913931437211 ], [ -1.744168717558851, 50.747455388760478 ], [ -1.742333282696719, 50.749248837499216 ], [ -1.743573663826132, 50.750007812390997 ], [ -1.743597060001076, 50.752715380358332 ], [ -1.744353417876372, 50.753608173620222 ], [ -1.744656151006579, 50.756072743732638 ], [ -1.742016029475893, 50.756667703514346 ], [ -1.741824020422771, 50.758315555802959 ], [ -1.741413666302129, 50.758686923050227 ], [ -1.741683957048484, 50.760338637565127 ], [ -1.740118049665699, 50.760963791151106 ], [ -1.739876920720833, 50.762773426276333 ], [ -1.738959623458475, 50.763270533455646 ], [ -1.739679702051122, 50.76534767899188 ], [ -1.741240609963547, 50.767207217362724 ], [ -1.744786114162262, 50.773116879709484 ], [ -1.746387080846646, 50.773403615200181 ], [ -1.748289466688452, 50.77514065389542 ], [ -1.745689478099918, 50.776256436163528 ], [ -1.749058955410211, 50.779510123094518 ], [ -1.752076485807119, 50.778845570785499 ], [ -1.756557845896551, 50.778419770891972 ], [ -1.767575638016281, 50.772912464493629 ], [ -1.768020486711846, 50.773304605113665 ], [ -1.77011733009873, 50.772548832077447 ], [ -1.769936281059312, 50.771315549361361 ], [ -1.768881558440415, 50.769782948421252 ], [ -1.773161868724678, 50.768189711169057 ], [ -1.775599566648192, 50.766992209877131 ], [ -1.776064893278307, 50.766352847321919 ], [ -1.77904769561578, 50.767365732248123 ], [ -1.779990854665754, 50.767338761618184 ], [ -1.781277226169933, 50.766669473803844 ], [ -1.782253379242102, 50.765021134156932 ], [ -1.783424884639047, 50.764649269489389 ], [ -1.783879103592941, 50.764845291008044 ], [ -1.783766320763572, 50.765624680742469 ], [ -1.784120838486509, 50.765919402731321 ], [ -1.785187545023539, 50.765840485620195 ], [ -1.78501005904059, 50.764950788466741 ], [ -1.78549590627468, 50.764752031910838 ], [ -1.785986019964536, 50.765487684615877 ], [ -1.788276830038308, 50.766869523909726 ], [ -1.788834034187079, 50.767832743770228 ], [ -1.787242347877778, 50.770490792128967 ], [ -1.787692805521534, 50.770603998222661 ], [ -1.789557739039627, 50.769947362718121 ], [ -1.789687982207484, 50.770288412830709 ], [ -1.788501433132982, 50.771151359493437 ], [ -1.790171452891285, 50.770969120846956 ], [ -1.790500210790918, 50.771657635988348 ], [ -1.789543172381538, 50.772233237872719 ], [ -1.788882420007956, 50.773482011440805 ], [ -1.788118203511641, 50.773759396633615 ], [ -1.787866346058172, 50.774249074312181 ], [ -1.788165891162733, 50.774489657967713 ], [ -1.789956865087159, 50.774558637121196 ], [ -1.790749314127242, 50.776212904553589 ], [ -1.790499747247321, 50.776533495457521 ], [ -1.788729234298345, 50.776943024343609 ], [ -1.788529904958341, 50.778075792189455 ], [ -1.787868245512759, 50.778855116695709 ], [ -1.790980662799374, 50.780332018346961 ], [ -1.792123436443487, 50.782665866218345 ], [ -1.794345309204769, 50.782208535361391 ], [ -1.795716247716975, 50.783357567382751 ], [ -1.797522762262186, 50.78383640269255 ], [ -1.797321224579245, 50.784520388391876 ], [ -1.795836141866298, 50.785454840652676 ], [ -1.79567677294204, 50.785901555651179 ], [ -1.796215287660833, 50.786353901852223 ], [ -1.79763902049944, 50.785819494226466 ], [ -1.798587889472772, 50.785853509066328 ], [ -1.800804079805107, 50.78742118496443 ], [ -1.802615554131834, 50.790809194412049 ], [ -1.801255147305437, 50.791117126992141 ], [ -1.800965313451324, 50.79188466605342 ], [ -1.804266036754354, 50.791371361736445 ], [ -1.805967199195137, 50.792020795939536 ], [ -1.805604830498796, 50.792524721061156 ], [ -1.803513131945984, 50.791945644690223 ], [ -1.803564207308453, 50.792963695123134 ], [ -1.800867886015067, 50.794118301994942 ], [ -1.801444723742508, 50.794615640363503 ], [ -1.803956539449077, 50.794817784008032 ], [ -1.804797204177895, 50.795684330518398 ], [ -1.803904959924503, 50.795965209735058 ], [ -1.803021937266247, 50.796080587988129 ], [ -1.802155845476028, 50.796897495086263 ], [ -1.803442870100186, 50.797558844296042 ], [ -1.803491342044769, 50.798529241913805 ], [ -1.801933013426845, 50.798944728263983 ], [ -1.801373459698769, 50.799714504323319 ], [ -1.802348643289159, 50.800318692927597 ], [ -1.804034302052185, 50.799987834143387 ], [ -1.804658152498108, 50.800459253548055 ], [ -1.804795059304087, 50.800989187437267 ], [ -1.804006488236345, 50.802589466875588 ], [ -1.804281887200228, 50.803578226965605 ], [ -1.806964276899167, 50.803506300916176 ], [ -1.809735500340251, 50.804956862983431 ], [ -1.810576138218347, 50.805901581465655 ], [ -1.811663868996671, 50.806076053387159 ], [ -1.812067891904662, 50.80654788536949 ], [ -1.811903993543113, 50.80709440478244 ], [ -1.810599630096837, 50.807098577026927 ], [ -1.811806564067787, 50.808018709836965 ], [ -1.811832150951621, 50.808730057265606 ], [ -1.810538567358517, 50.808893462672643 ], [ -1.809938933221481, 50.809770196374039 ], [ -1.806761085161084, 50.810466366057156 ], [ -1.805733045696392, 50.811627353217666 ], [ -1.802873599294014, 50.813512003257649 ], [ -1.803491710668624, 50.814052552517595 ], [ -1.806424408070303, 50.81501160919521 ], [ -1.807273698611976, 50.816299878146239 ], [ -1.806593147919667, 50.817831125038943 ], [ -1.805257976564151, 50.818362192544654 ], [ -1.804026135489517, 50.819984199077062 ], [ -1.802047412910069, 50.820611234875663 ], [ -1.80175630048349, 50.820975830260359 ], [ -1.801668691213077, 50.82223646484649 ], [ -1.802137686367035, 50.822437836889236 ], [ -1.803215987057651, 50.821929745242535 ], [ -1.804039194096818, 50.821959906189562 ], [ -1.804525859279709, 50.822360969338078 ], [ -1.804325125537961, 50.822836302671242 ], [ -1.804845405926851, 50.823713991700075 ], [ -1.804579994628264, 50.824709928175338 ], [ -1.8032389614098, 50.825246355162619 ], [ -1.804624304256752, 50.827030189466136 ], [ -1.8036350079783, 50.830023959825361 ], [ -1.802927699652606, 50.830741256656076 ], [ -1.801366346852213, 50.830952653268824 ], [ -1.80056313328431, 50.831498059552885 ], [ -1.798953481428298, 50.831388211423892 ], [ -1.797976694857704, 50.83036316489288 ], [ -1.797144625538486, 50.830020896664969 ], [ -1.795660339191002, 50.830789023399753 ], [ -1.794436814809993, 50.831029598797791 ], [ -1.794117988642295, 50.831518280194551 ], [ -1.795770925586399, 50.831822458754829 ], [ -1.796655103824494, 50.833221442578974 ], [ -1.795502064527162, 50.833877679215611 ], [ -1.79206692590921, 50.833588300337212 ], [ -1.791533568711938, 50.834102661129414 ], [ -1.791627494700992, 50.835335741643256 ], [ -1.790614920444223, 50.836314119430263 ], [ -1.791391742375245, 50.836956696128148 ], [ -1.79294126319294, 50.836589882849054 ], [ -1.79440579748942, 50.836801145555484 ], [ -1.794545189204206, 50.837724053964664 ], [ -1.796788521109752, 50.83815512319309 ], [ -1.798564962648799, 50.838851539070212 ], [ -1.800005862691056, 50.839988048675423 ], [ -1.800322330604305, 50.841043408981747 ], [ -1.79926605442303, 50.841301494520636 ], [ -1.798668703272573, 50.840817508348984 ], [ -1.79735521724396, 50.84079095097227 ], [ -1.796995944941826, 50.841092483196014 ], [ -1.798077126103237, 50.841987360928535 ], [ -1.799617275718803, 50.841538631554101 ], [ -1.799848788812677, 50.842195492376931 ], [ -1.801873103505588, 50.842420211110209 ], [ -1.803358585317261, 50.844476624773911 ], [ -1.803610870458287, 50.845279242789012 ], [ -1.802745320289505, 50.845418022565433 ], [ -1.800219248187863, 50.847240144823374 ], [ -1.800409797900313, 50.84787176046548 ], [ -1.799942856599373, 50.848449203366243 ], [ -1.801271638844552, 50.849284151714507 ], [ -1.80237005818813, 50.849543232789927 ], [ -1.802883849331178, 50.850661040059379 ], [ -1.804805061430804, 50.851154334154373 ], [ -1.805497656646333, 50.851990946695992 ], [ -1.805838472289934, 50.853070600155903 ], [ -1.803911312551883, 50.853250855734913 ], [ -1.803247280221921, 50.853760554352718 ], [ -1.804755882203278, 50.854781046394052 ], [ -1.804089278192361, 50.855538872174087 ], [ -1.80400301875641, 50.85647576752374 ], [ -1.804841337473789, 50.856791918985472 ], [ -1.805434530025232, 50.857636452651128 ], [ -1.805086422305302, 50.859009918660597 ], [ -1.805565724128616, 50.859218478406802 ], [ -1.807174528883349, 50.860138401304383 ], [ -1.806413086200664, 50.861497689766573 ], [ -1.808027784506221, 50.86445986979232 ], [ -1.810958249683013, 50.864335105764312 ], [ -1.811373461735441, 50.864593880431904 ], [ -1.812790202182919, 50.864572786474263 ], [ -1.813749888131918, 50.8633809898542 ], [ -1.812924283198884, 50.862332065541111 ], [ -1.814745449116484, 50.862051696119529 ], [ -1.814832627356362, 50.858701171627274 ], [ -1.821236170178452, 50.856772367062796 ], [ -1.82951847980606, 50.855318931570899 ], [ -1.841631626685448, 50.858042777130855 ], [ -1.846019971822223, 50.858136743157665 ], [ -1.850943733562716, 50.858724957750901 ], [ -1.850563740145434, 50.860298167683652 ], [ -1.853208821883505, 50.862672894134441 ], [ -1.853538834728633, 50.863460110641256 ], [ -1.851665120995008, 50.864223063622738 ], [ -1.853539262309691, 50.866505013098894 ], [ -1.847832182060654, 50.86968646276371 ], [ -1.848479764472532, 50.873778003636986 ], [ -1.845272309121279, 50.877997649227822 ], [ -1.846121941869021, 50.879420446320509 ], [ -1.847710559471228, 50.880877555662053 ], [ -1.847977235525254, 50.882339134082791 ], [ -1.84749193607603, 50.883407769879305 ], [ -1.844490508174651, 50.885296749827404 ], [ -1.844043280980344, 50.886829401154536 ], [ -1.845099105604075, 50.888250688506872 ], [ -1.848555532184115, 50.8898873570296 ], [ -1.844889367632525, 50.893168486578084 ], [ -1.841707165471136, 50.897020214759209 ], [ -1.839869880057666, 50.897890012354686 ], [ -1.837238848471888, 50.897197561720446 ], [ -1.831473662944018, 50.897390764053569 ], [ -1.825575848390247, 50.896167197603326 ], [ -1.824953172357212, 50.895770614990525 ], [ -1.823898765726265, 50.898225820523471 ], [ -1.816541926814204, 50.903948145812073 ], [ -1.817272728097802, 50.905428596093032 ], [ -1.817569174061655, 50.907464935002373 ], [ -1.820407165073362, 50.912105919725462 ], [ -1.819158808514637, 50.912700209568712 ], [ -1.81861110474013, 50.914545542611449 ], [ -1.818843844029567, 50.916526036444701 ], [ -1.817481075338599, 50.917608466548671 ], [ -1.815033877666708, 50.920583805211315 ], [ -1.814467103167377, 50.922882360902094 ], [ -1.81477344653144, 50.923131869422249 ], [ -1.812603320877994, 50.925382012684672 ], [ -1.810793171699444, 50.926525581504826 ], [ -1.810547074323224, 50.926865990482128 ], [ -1.811059978604541, 50.927036821468427 ], [ -1.810658089669101, 50.927203383384814 ], [ -1.811590760419912, 50.927371296101725 ], [ -1.811438144249041, 50.927803555328573 ], [ -1.812783954563332, 50.927519751562265 ], [ -1.813215230832779, 50.927835180614451 ], [ -1.813926952167308, 50.92704951758045 ], [ -1.814283304266392, 50.927237974794046 ], [ -1.815000402628718, 50.926893809354567 ], [ -1.815358004567388, 50.927122809951378 ], [ -1.817465047261889, 50.927160285195612 ], [ -1.817850775727576, 50.926769734731778 ], [ -1.818534834379046, 50.926858001778655 ], [ -1.820747974326834, 50.926299365134135 ], [ -1.822391698413939, 50.926587845975128 ], [ -1.822647887185473, 50.926932640399478 ], [ -1.822912256752467, 50.92663817055135 ], [ -1.823248128673142, 50.926976791413367 ], [ -1.823598997586171, 50.926763270378558 ], [ -1.824142663279855, 50.927095010837739 ], [ -1.824802485144965, 50.926831606900947 ], [ -1.826300543724179, 50.926869803620413 ], [ -1.8282720776596, 50.927768388182962 ], [ -1.829741225681364, 50.927572668859391 ], [ -1.832622381730663, 50.929199160118934 ], [ -1.834451680413565, 50.929345605322169 ], [ -1.835380052468035, 50.929570857164038 ], [ -1.83538971565788, 50.930057317639616 ], [ -1.838749436100877, 50.930803044620454 ], [ -1.840319503573443, 50.931883366896635 ], [ -1.840721907973786, 50.93158086493785 ], [ -1.842013753741809, 50.931620376404005 ], [ -1.84265915516542, 50.930960304800308 ], [ -1.844137040296226, 50.930724860736746 ], [ -1.844972636156034, 50.930188282391967 ], [ -1.846049457617069, 50.930280504466744 ], [ -1.846595370756897, 50.929561845482034 ], [ -1.847145720216317, 50.929668647994099 ], [ -1.851057130206081, 50.928834665116739 ], [ -1.855957838659101, 50.926457851399896 ], [ -1.855813778277628, 50.926115964803849 ], [ -1.856479572580451, 50.925680592140111 ], [ -1.855970649608473, 50.92553792375999 ], [ -1.855747481771651, 50.924982786768673 ], [ -1.857466140571593, 50.923152206485 ], [ -1.85909327787797, 50.922373674616388 ], [ -1.859904585100866, 50.921304486403365 ], [ -1.860940754362439, 50.921126826259808 ], [ -1.861070297352696, 50.920639544220414 ], [ -1.863488185821374, 50.919388837688764 ], [ -1.864631872062134, 50.918923500717227 ], [ -1.873644321449642, 50.917225889786508 ], [ -1.875007393215778, 50.91979660803036 ], [ -1.876117861722427, 50.920585497085462 ], [ -1.877871108342871, 50.923105263233303 ], [ -1.879230189803098, 50.924064383544042 ], [ -1.878802628739695, 50.924325637336764 ], [ -1.880400160107373, 50.924956767740454 ], [ -1.881578926192736, 50.92637695120596 ], [ -1.892553970724881, 50.931623993295183 ], [ -1.896899382805131, 50.934395844806488 ], [ -1.898566876688832, 50.935878406088456 ], [ -1.899086529663251, 50.938392269197379 ], [ -1.902599861529973, 50.940726089529257 ], [ -1.905381783019111, 50.943083571941912 ], [ -1.909183490937204, 50.945326623000035 ], [ -1.910576604940535, 50.949223212895085 ], [ -1.912442369493573, 50.951015959350364 ], [ -1.912484282737414, 50.952198521071765 ], [ -1.916105723033404, 50.953131011829967 ], [ -1.918740914123718, 50.956365643238698 ], [ -1.920986085908541, 50.961495508196073 ], [ -1.923923002847092, 50.962723122884185 ], [ -1.927183854957461, 50.963594784302487 ], [ -1.928402046364806, 50.964988405206164 ], [ -1.928684889896733, 50.966293345328559 ], [ -1.929888826076993, 50.967795803730525 ], [ -1.934864235916746, 50.969092664805515 ], [ -1.939219935996093, 50.969657054014668 ], [ -1.941258011176699, 50.971826134675915 ], [ -1.942079275715759, 50.97336332673359 ], [ -1.944829646519326, 50.975748447468114 ], [ -1.948133302969429, 50.976474802517899 ], [ -1.950797909405735, 50.978129605581081 ], [ -1.952521053215147, 50.978582591003367 ], [ -1.954729349342955, 50.978117718916224 ], [ -1.95546558071481, 50.978266332705246 ], [ -1.957224495152958, 50.983194751347568 ], [ -1.956820894980916, 50.989829933313594 ], [ -1.962636669030962, 50.991853443661959 ], [ -1.965357919504081, 50.992036776635395 ], [ -1.968056113811717, 50.990571884258578 ], [ -1.971989222236102, 50.989523453036149 ], [ -1.972979133995462, 50.987532847321226 ], [ -1.975247641484388, 50.986910193434788 ], [ -1.977372326773418, 50.985769485441075 ], [ -1.97947241521707, 50.985573904070378 ], [ -1.980421430080367, 50.984921241556314 ], [ -1.983381967041123, 50.984434289102879 ], [ -1.985209659347501, 50.984755555903028 ], [ -1.986049008991766, 50.984051592490836 ], [ -1.985885385711691, 50.983268306587107 ], [ -1.986899876603111, 50.982375485133922 ], [ -1.989790503694028, 50.981560235750266 ], [ -1.990101179683963, 50.980922718569261 ], [ -1.989588489976718, 50.980449639273822 ], [ -1.989228391944492, 50.978860690205344 ], [ -1.990285477336185, 50.978503805838656 ], [ -1.991220116071582, 50.977144289542522 ], [ -1.992252824186937, 50.976862897778055 ], [ -1.992992229158845, 50.976057205419643 ], [ -1.994036299182692, 50.97572815359765 ], [ -1.999048555843409, 50.975265252340066 ], [ -2.00066799836302, 50.974780534990138 ], [ -2.002702001241524, 50.974844419097401 ], [ -2.003812923656698, 50.974509790435782 ], [ -2.004979431102587, 50.974632991122988 ], [ -2.007060537405042, 50.975448415946417 ], [ -2.009719656085043, 50.975257637218675 ], [ -2.012300288826011, 50.974165687747508 ], [ -2.014010880612269, 50.97388941838873 ], [ -2.014872251361386, 50.973149284906512 ], [ -2.017419916266879, 50.972158000709584 ], [ -2.020279831167013, 50.972226751166055 ], [ -2.021949286578581, 50.9726148864046 ], [ -2.023723845650672, 50.972511153907561 ], [ -2.028967091945877, 50.971179131764323 ], [ -2.032854182490912, 50.971851594721386 ], [ -2.034454115191832, 50.972424846926607 ], [ -2.037208189785978, 50.971969838880149 ], [ -2.038338561823103, 50.971409325419835 ], [ -2.03853880764999, 50.970749239227949 ], [ -2.04122533417257, 50.968067689393514 ], [ -2.051078117024223, 50.966151991587267 ], [ -2.051556815393953, 50.966363129974113 ], [ -2.05331966022464, 50.966233680037057 ], [ -2.058327692732585, 50.963643247765731 ], [ -2.059404560998667, 50.962812662533814 ], [ -2.058427300671481, 50.961340264756998 ], [ -2.058600165070446, 50.960631505389884 ], [ -2.059318173264948, 50.959789506101863 ], [ -2.062372550114937, 50.959886778724176 ], [ -2.062532278899277, 50.959061152723592 ], [ -2.064272429315925, 50.957185271995513 ], [ -2.064861674008252, 50.95396208484479 ], [ -2.067283111249882, 50.951820490145664 ], [ -2.068312879469842, 50.951194014757661 ], [ -2.082863640836099, 50.951338364756083 ], [ -2.083883672578936, 50.950968061315407 ], [ -2.085269861552206, 50.949244964583968 ], [ -2.087369555468332, 50.949951083527537 ], [ -2.089219157490253, 50.950155636378859 ], [ -2.090916156183117, 50.949507774535313 ], [ -2.094145345390267, 50.949134734099246 ], [ -2.100253424817661, 50.94507045455665 ], [ -2.101791951105631, 50.944985512462466 ], [ -2.102860960733713, 50.945607747676 ], [ -2.102655248682606, 50.946570110940371 ], [ -2.103958689365311, 50.948916825752733 ], [ -2.105556944873161, 50.950008908678178 ], [ -2.110521176120261, 50.955488716012496 ], [ -2.112464325394943, 50.956570486290964 ], [ -2.115641572769496, 50.9575034818793 ], [ -2.119461601801886, 50.959680192238345 ], [ -2.120670946503727, 50.961539474687235 ], [ -2.121523326761035, 50.964015098031503 ], [ -2.121513206953964, 50.965034820829395 ], [ -2.12087838430051, 50.965715380526341 ], [ -2.12065165846279, 50.969407809449621 ], [ -2.123156891509621, 50.969468202344935 ], [ -2.122973878765813, 50.971873789364807 ], [ -2.122454666325624, 50.973758232773385 ], [ -2.119764358950043, 50.978190648638815 ], [ -2.121966365016183, 50.978167680053822 ], [ -2.122343570454057, 50.978604364659155 ], [ -2.126853692929681, 50.978738867659743 ], [ -2.131689714994851, 50.979770271388752 ], [ -2.138396423634322, 50.982287504129509 ], [ -2.143577247959259, 50.984964552817907 ], [ -2.148147245719886, 50.986277962596006 ], [ -2.152523315808576, 50.989157906098363 ], [ -2.159638255256252, 50.992782181757327 ], [ -2.161842035289124, 50.993475120862115 ], [ -2.161526521589726, 50.994129290438678 ], [ -2.163613576140734, 50.994412338189029 ], [ -2.163582755317623, 50.996166812193501 ], [ -2.164436027798228, 50.996084702121657 ], [ -2.167082277674907, 50.998083480695193 ], [ -2.169306237668524, 51.000661033211905 ], [ -2.171798539543152, 51.001000885045329 ], [ -2.172565514120441, 51.001072569416358 ], [ -2.172969999889184, 51.00215468870659 ], [ -2.17608778126871, 51.005403392913742 ], [ -2.176414661306574, 51.006658218558613 ], [ -2.178101314274134, 51.00826076349145 ], [ -2.176323504939296, 51.011519616483753 ], [ -2.176388266734993, 51.012817987796218 ], [ -2.178761034689815, 51.014132621594563 ], [ -2.185176840166688, 51.015775362225831 ], [ -2.183822137473857, 51.017482486473575 ], [ -2.185991753774414, 51.018151654430156 ], [ -2.188603767408163, 51.018532307915514 ], [ -2.188836422725751, 51.025269863711884 ], [ -2.190137815582562, 51.026551831297802 ], [ -2.190319502144017, 51.027394944384859 ], [ -2.190288433328003, 51.028506475054101 ], [ -2.189339897990839, 51.029496251042033 ], [ -2.189183001593684, 51.030522493479872 ], [ -2.190548518319294, 51.032120876313087 ], [ -2.19199595254479, 51.036246815089243 ], [ -2.19320866066048, 51.037712370956449 ], [ -2.194718078887043, 51.038852803139079 ], [ -2.195170804957502, 51.039657697880713 ], [ -2.198124631176048, 51.041654440126266 ], [ -2.201531323210401, 51.042768083098153 ], [ -2.201434973351466, 51.043249374415467 ], [ -2.205322894069157, 51.042793836672843 ], [ -2.207216343094449, 51.04581370168512 ], [ -2.210614836845505, 51.047182491002083 ], [ -2.21243993632258, 51.050111597192306 ], [ -2.215113838607712, 51.051736967973845 ], [ -2.217071396310267, 51.053610914388429 ], [ -2.217993377199087, 51.055178332249149 ], [ -2.223269117995658, 51.056562110456625 ], [ -2.225808117882661, 51.057582401084623 ], [ -2.227779221730432, 51.06030404123215 ], [ -2.228891647877775, 51.060784786853738 ], [ -2.236838809123415, 51.06672895329767 ], [ -2.238825662414618, 51.067874046798309 ], [ -2.238983701195227, 51.068651592980984 ], [ -2.240164818528735, 51.069906275044779 ], [ -2.244689005665542, 51.072310409303689 ], [ -2.247645827838527, 51.072206099830062 ], [ -2.251309527953667, 51.070836888700406 ], [ -2.268620138662786, 51.068740908339095 ], [ -2.283112463272514, 51.075146033732807 ], [ -2.291269285808318, 51.077075475333672 ], [ -2.29326546139102, 51.076939236708562 ], [ -2.298681240481638, 51.078486566464498 ], [ -2.300211821035987, 51.078546472051165 ], [ -2.303349916939054, 51.079262212134502 ], [ -2.306434037716056, 51.079557208143733 ], [ -2.309471955267604, 51.079150876408079 ], [ -2.3108798620286, 51.080262120302905 ], [ -2.313299915148257, 51.079884299830738 ], [ -2.315705127746354, 51.080044226608891 ], [ -2.316053058391606, 51.079786094505238 ], [ -2.316805819489742, 51.07984338821079 ] ] ], [ [ [ -1.971843196124786, 50.696123707483117 ], [ -1.974168874264219, 50.697227723871627 ], [ -1.976298393002655, 50.697219247626947 ], [ -1.978726955347967, 50.696725156876482 ], [ -1.983548757891108, 50.694878934085096 ], [ -1.98484147771599, 50.694698389722717 ], [ -1.985594646904533, 50.694935031954877 ], [ -1.987421605293167, 50.692665404280291 ], [ -1.98808847416946, 50.692453282378551 ], [ -1.985161904933146, 50.68845737045767 ], [ -1.983545738920224, 50.687074920512593 ], [ -1.981878115646549, 50.687189750239042 ], [ -1.980353642070223, 50.686918782160561 ], [ -1.9747859636027, 50.687245855629222 ], [ -1.970586414945138, 50.686349994866525 ], [ -1.96664723483653, 50.685915421599887 ], [ -1.96105959631957, 50.686289426011925 ], [ -1.958299135147344, 50.687871119080292 ], [ -1.956764264658363, 50.688219433587477 ], [ -1.957129248368793, 50.688483977540564 ], [ -1.956089759376112, 50.688906215167641 ], [ -1.956488713178742, 50.689189634457193 ], [ -1.955914913346468, 50.691110236877137 ], [ -1.95748907163471, 50.694329440179715 ], [ -1.958155445183178, 50.69493042263251 ], [ -1.961294151691292, 50.695252633947057 ], [ -1.964999800315319, 50.694782648513652 ], [ -1.971843196124786, 50.696123707483117 ] ] ], [ [ [ -2.018010979578318, 50.694062068602058 ], [ -2.018676445625592, 50.69406913712146 ], [ -2.019443907742339, 50.694286614565364 ], [ -2.021032113249404, 50.693545303325848 ], [ -2.022004999760655, 50.693948847874331 ], [ -2.019383026246445, 50.691007827300808 ], [ -2.019532225804551, 50.689034766779052 ], [ -2.020403930973013, 50.688169485815628 ], [ -2.019842612050644, 50.686598534418785 ], [ -2.018915070073083, 50.685729067657867 ], [ -2.019152707961628, 50.685432283718377 ], [ -2.018684105904651, 50.685224663152638 ], [ -2.018628660536586, 50.684660797474599 ], [ -2.018017158120703, 50.684753506399957 ], [ -2.01738002416466, 50.684426275022489 ], [ -2.01439629642679, 50.68513441103476 ], [ -2.014043996325684, 50.685490605363711 ], [ -2.014259382561808, 50.686181233787956 ], [ -2.013312522001905, 50.686728983135467 ], [ -2.013857628931296, 50.687023919665087 ], [ -2.013038216764299, 50.687802816134464 ], [ -2.012234176791832, 50.687965640800314 ], [ -2.013836765767995, 50.688126464151424 ], [ -2.013978366819888, 50.688459170072761 ], [ -2.014623889987606, 50.688299902322299 ], [ -2.014348000319324, 50.688799953176435 ], [ -2.015071339958046, 50.688604707372122 ], [ -2.015296210083878, 50.687982375651202 ], [ -2.015719482868792, 50.688145091067348 ], [ -2.015285068378596, 50.688525540293362 ], [ -2.01572130724419, 50.689082145466379 ], [ -2.01695707311668, 50.688859837575507 ], [ -2.017846180355901, 50.689282372546622 ], [ -2.01627483712107, 50.689136915321015 ], [ -2.015817811308356, 50.689922954015628 ], [ -2.015544525896916, 50.689633437079685 ], [ -2.014768885448055, 50.690116450524073 ], [ -2.014607756677831, 50.690894370518016 ], [ -2.015332809937481, 50.691571448718392 ], [ -2.015952894080053, 50.691464331060352 ], [ -2.017239973724561, 50.691977622330711 ], [ -2.018239980278934, 50.693108779163829 ], [ -2.018010979578318, 50.694062068602058 ] ] ], [ [ [ -1.993857192587035, 50.682874438664911 ], [ -1.994845211646924, 50.682799849812582 ], [ -1.995602563888216, 50.681860122375141 ], [ -1.996603324162423, 50.681325051910399 ], [ -1.99670385300853, 50.679970741884041 ], [ -1.996266505200389, 50.679675758164088 ], [ -1.995797997300992, 50.679977004344352 ], [ -1.995366417116493, 50.678805210615607 ], [ -1.994296523893974, 50.677765584574246 ], [ -1.991400859218992, 50.677702455910023 ], [ -1.990629625878327, 50.677058502492294 ], [ -1.988529007125801, 50.678740894480363 ], [ -1.987955833155537, 50.678712967406717 ], [ -1.98710789633152, 50.679270442984468 ], [ -1.988054709341006, 50.679653640591553 ], [ -1.988826193588064, 50.678883001385479 ], [ -1.988746741444942, 50.679545824063943 ], [ -1.989611641794508, 50.679167262266034 ], [ -1.989182661147546, 50.679820070943137 ], [ -1.988361697961293, 50.679829881437321 ], [ -1.989648195960078, 50.680339934600759 ], [ -1.989968161915869, 50.67990830967404 ], [ -1.990538542559805, 50.679910180480256 ], [ -1.989948220068492, 50.680567490642112 ], [ -1.990866719234148, 50.681114374954994 ], [ -1.991558917286945, 50.68079517830548 ], [ -1.991536160660232, 50.681277157276142 ], [ -1.99197344752736, 50.68147415902623 ], [ -1.992671401189492, 50.680860881911244 ], [ -1.995344998233409, 50.680628077070985 ], [ -1.994167297562571, 50.681546210095519 ], [ -1.994618806585092, 50.681613659488839 ], [ -1.994256457742255, 50.682025528716977 ], [ -1.994798548306586, 50.682521061516468 ], [ -1.993946474112817, 50.682392428493223 ], [ -1.993857192587035, 50.682874438664911 ] ] ], [ [ [ -1.986159571482712, 50.684658887356917 ], [ -1.990551870455808, 50.684318511843472 ], [ -1.990559054679927, 50.683957010463168 ], [ -1.991140789830051, 50.684150355009947 ], [ -1.991047464542648, 50.683440816976045 ], [ -1.990131638496992, 50.683659273596788 ], [ -1.989899696518245, 50.682631361625511 ], [ -1.988989834031014, 50.681673504598983 ], [ -1.98721194196811, 50.68196561192272 ], [ -1.9846472630411, 50.681813346129637 ], [ -1.982191381393029, 50.682026943401269 ], [ -1.980046464659488, 50.683523015112399 ], [ -1.981351144397187, 50.684495390669319 ], [ -1.982138391632442, 50.683894800318122 ], [ -1.981685337409784, 50.68426886503044 ], [ -1.983175728501785, 50.68449929301083 ], [ -1.984821989895424, 50.684300828013413 ], [ -1.986159571482712, 50.684658887356917 ] ] ], [ [ [ -1.998941512157309, 50.683115612131942 ], [ -1.999837476432286, 50.683318890633132 ], [ -1.999770969298045, 50.683830540003079 ], [ -2.001030783733825, 50.684835951742471 ], [ -2.002948806814975, 50.683891617763983 ], [ -2.002175870138209, 50.683706420605198 ], [ -2.002365525155711, 50.682694748016502 ], [ -2.002743416376868, 50.68282509610431 ], [ -2.003254287430833, 50.681652402214723 ], [ -2.002044080099749, 50.681383547149721 ], [ -2.002942824675559, 50.680735121056699 ], [ -2.002753133855621, 50.680337647073223 ], [ -2.001356125411153, 50.680230679374965 ], [ -2.000965499981194, 50.680567004407131 ], [ -2.000798455909818, 50.68094111576368 ], [ -2.00122873719126, 50.681177648385869 ], [ -2.000386643289571, 50.681665021995009 ], [ -2.000303106273544, 50.682464529558182 ], [ -1.999479417179642, 50.682627261176435 ], [ -1.998941512157309, 50.683115612131942 ] ] ], [ [ [ -1.770281574293118, 50.73328219204533 ], [ -1.771544372207144, 50.734961794488129 ], [ -1.771164098823379, 50.735074320822726 ], [ -1.771744330299156, 50.736389228097423 ], [ -1.77202437612067, 50.736486947200447 ], [ -1.773702393043831, 50.735275340128638 ], [ -1.773882289951786, 50.734700153831213 ], [ -1.772318143654735, 50.732899482317258 ], [ -1.769691455405379, 50.731095762706857 ], [ -1.770281574293118, 50.73328219204533 ] ] ], [ [ [ -1.993632816743109, 50.677303309630631 ], [ -1.994915021880077, 50.677712548599075 ], [ -1.995099075214205, 50.67695714839148 ], [ -1.994627842974818, 50.676071280523111 ], [ -1.993006070107679, 50.675549658700092 ], [ -1.991173230555579, 50.676286951055665 ], [ -1.993843741550653, 50.676835686938574 ], [ -1.993440363500348, 50.677061389199011 ], [ -1.9913967651107, 50.67675549764742 ], [ -1.991079708801785, 50.67693891850282 ], [ -1.991446167163718, 50.677568466240182 ], [ -1.992025031617639, 50.677547813717467 ], [ -1.991996764083614, 50.677234864826744 ], [ -1.993632816743109, 50.677303309630631 ] ] ], [ [ [ -2.004771434671826, 50.679859145861052 ], [ -2.005721153351813, 50.679947234827011 ], [ -2.005201768416817, 50.680314168281861 ], [ -2.005644820286684, 50.680654075141724 ], [ -2.005358945669893, 50.680967943222022 ], [ -2.006579105954279, 50.6813069072617 ], [ -2.007078855039319, 50.682165711830145 ], [ -2.007582833837268, 50.682341028206643 ], [ -2.00833015404439, 50.682047806042966 ], [ -2.007391642985148, 50.681740312711227 ], [ -2.006522332961447, 50.680291612533253 ], [ -2.005967544872864, 50.680279962319617 ], [ -2.00649254747043, 50.679890525675276 ], [ -2.00628581938491, 50.679286213655772 ], [ -2.005214286822636, 50.678831205551475 ], [ -2.004771434671826, 50.679859145861052 ] ] ], [ [ [ -1.741118319098391, 50.721533395085594 ], [ -1.74037918537969, 50.722492178035331 ], [ -1.740788208856426, 50.723601025274661 ], [ -1.742244028737076, 50.722928913075201 ], [ -1.742798990196103, 50.722218741327573 ], [ -1.742690787489144, 50.721531470461009 ], [ -1.741118319098391, 50.721533395085594 ] ] ], [ [ [ -2.004589369924992, 50.684721633635228 ], [ -2.004505973276092, 50.685325966096769 ], [ -2.005639754402328, 50.684678417701889 ], [ -2.006824531424923, 50.684737688571488 ], [ -2.009645536489616, 50.684100776981438 ], [ -2.008592307520438, 50.683501035456565 ], [ -2.007860541480004, 50.684010118997719 ], [ -2.007985211288759, 50.684346426169434 ], [ -2.00672545339451, 50.684611802142442 ], [ -2.006511589260834, 50.683975119169581 ], [ -2.00594821715778, 50.683948172534429 ], [ -2.005155463439224, 50.683251232667722 ], [ -2.005134316518825, 50.684074119617904 ], [ -2.004589369924992, 50.684721633635228 ] ] ], [ [ [ -1.977819549800229, 50.665753503417939 ], [ -1.978605845464788, 50.666552210631927 ], [ -1.979191667261421, 50.666575717921589 ], [ -1.98055295675341, 50.666224331922244 ], [ -1.980667792505757, 50.665793615303848 ], [ -1.980206602558204, 50.665531825109937 ], [ -1.979103014080625, 50.665430004235333 ], [ -1.977819549800229, 50.665753503417939 ] ] ], [ [ [ -2.072813928361225, 50.692366378308414 ], [ -2.073393204994297, 50.692484761479896 ], [ -2.074411332784371, 50.691735899979903 ], [ -2.073467125195146, 50.690884880028896 ], [ -2.072731670893696, 50.691340379530899 ], [ -2.072379385599239, 50.690565423881459 ], [ -2.071954257700695, 50.691205078908986 ], [ -2.072477379025826, 50.691675966592051 ], [ -2.073233286740131, 50.691626038950623 ], [ -2.072813928361225, 50.692366378308414 ] ] ], [ [ [ -2.009496802027354, 50.683599888598515 ], [ -2.010851575132561, 50.68405929704987 ], [ -2.011884807243379, 50.683586125293971 ], [ -2.010358725118702, 50.683070135444012 ], [ -2.009687787453159, 50.683109761193606 ], [ -2.009634067589208, 50.683445198845533 ], [ -2.009226284543205, 50.68296407125905 ], [ -2.009496802027354, 50.683599888598515 ] ] ], [ [ [ -1.803548415011478, 50.740140536155565 ], [ -1.804588741076171, 50.739759179801126 ], [ -1.804432561664697, 50.73949540592465 ], [ -1.801184259885797, 50.739204869092539 ], [ -1.803548415011478, 50.740140536155565 ] ] ], [ [ [ -2.071283812742647, 50.68973335971485 ], [ -2.072560496581422, 50.690505054487794 ], [ -2.073006310353571, 50.690427423253247 ], [ -2.072193040120497, 50.689988195531761 ], [ -2.072380246874902, 50.689328924344295 ], [ -2.071283812742647, 50.68973335971485 ] ] ], [ [ [ -2.225647672797433, 50.616562200399208 ], [ -2.226281834160817, 50.616453892702168 ], [ -2.226024818041882, 50.616207144901125 ], [ -2.225647672797433, 50.616562200399208 ] ] ], [ [ [ -2.225213345856174, 50.616473080628829 ], [ -2.225397192157894, 50.616196670781029 ], [ -2.225007893416279, 50.616090341785053 ], [ -2.225213345856174, 50.616473080628829 ] ] ] ] } }

In [78]:
mapdata['E10000009'] = dorset


# Save Files

Files can be saved for later use, to avoid repeating the cleaning process and in particular the long and not always successful process of acquiring the map data.

In [69]:
import numpy as np
np.save('mapdata.npy', mapdata) 
df_merged.to_csv(r'Merged.csv', index=False)
df_deprivation2.to_csv(r'IMD.csv', index=False)
df_gcse.to_csv(r'KS4.csv', index=False)